# PRE-REQUISITES -

RESOURCES USED:
1. SQL server Management Studio 2020 + Sample DBs (Adventureworks)

STEPS TO ADD SAMPLE DBs: 
1. Download .Bak(backups) of sample DBs from web and restore in SQL server. 

ABOUT SAMPLE DBs: 
1. We will be using 2 DBs- AdventureWorks2022 and AdventureWorksDW2022.
2. The first one is an OLTP (Online Transaction Processing) database  and the DW one is an OLAP (Online Analytical Processing) database.
3. OLTP are optimised for Day-to-day transactions and OLAP is optimized for aggregations, BI, dashboards, and reporting.
4. OLTP → ETL → DW. 
5. OLTP = system of record and OLAP/DW = system of insight.


HANDY SHORTCUTS:
1. Comment the selected text	Ctrl+K and Ctrl+C
2. Uncomment the selected text	Ctrl+K and Ctrl+U

RECOMMENDED READING
1. First, second and  third Normal form

In [ ]:
NOTE: How is SQL different from other Programming languages?
1.  SQL is declarative, not procedural. In SQL, you describe what you want, not how to do it. You don’t tell the database:
how to loop through rows
how to filter them
how to select columns
The database engine figures all that out internally.
In procedural languages like Python, you explicitly write the logic step-by-step.

2. SQL is not general‑purpose
SQL cannot: build mobile apps, create games, write operating systems, control hardware or build server-side logic (by itself)
It’s specialized for interacting with relational databases

# SQL Order of Execution
1. FROM – Chooses the table to get data from.
2. WHERE – Filters rows.
3. GROUP BY – Groups rows.
4. SELECT – Calculates the total amount for each group.
5. WINDOW FUNCTIONS (OVER).
6. ORDER BY – Sorts the results.

# SQL JOINS
1. INNER JOIN
2. LEFT JOIN
3. FULL OUTER JOIN
4. EQUI AND NON-EQUI JOIN
5. SELF JOIN

In [ ]:
# QUESTION - What are the Geography associated with each customer. 

# Tables used - 
SELECT * FROM [AdventureWorksDW2022].[dbo].[DimCustomer]
SELECT * FROM [AdventureWorksDW2022].[dbo].[DimGeography]
# INNER JOIN - result includes only customers who have a matching geography row
# LEFT JOIN - show all customers, even if geography is missing

# Query - 

SELECT dc.CustomerKey, dc.FirstName, dc.LastName, dc.GeographyKey, dg.GeographyKey, dg.EnglishCountryRegionName, dg.StateProvinceCode, dg.City
	FROM DimCustomer as dc
	join DimGeography as dg on dc.GeographyKey = dg.GeographyKey

In [ ]:
# QUESTION - Find a list of all product subcategories and the category each one belongs to

# Tables used - 
SELECT * from DimProductCategory
SELECT * from DimProductSubcategory

# Query - 

Select dps.ProductSubcategoryKey,dps.EnglishProductSubcategoryName,dps.ProductCategoryKey
		,dpc.ProductCategoryKey,dpc.EnglishProductCategoryName
from DimProductCategory as dpc
join DimProductSubcategory as dps ON dpc.ProductCategoryKey= dps.ProductCategoryKey

In [ ]:
# QUESTION - Find complete product list with the subcategory and category each product belongs to

# Tables used - 
SELECT * from DimProductCategory
SELECT * from DimProductSubcategory
SELECT * from DimProduct

# Note: Checking these tables helps one understand the relationship between the tables. 

# Query - 

Select dp.ProductKey,dp.EnglishProductName,dp.ProductSubcategoryKey
		,dps.EnglishProductSubcategoryName,dps.ProductCategoryKey
		,dpc.ProductCategoryKey,dpc.EnglishProductCategoryName
from DimProduct as dp
join DimProductSubcategory as dps ON dp.ProductSubcategoryKey= dps.ProductSubcategoryKey
join DimProductCategory as dpc ON dpc.ProductCategoryKey= dps.ProductCategoryKey

In [ ]:
# QUESTION - Find a full detailed report of sales including product, customer, geography, currency, and territory information
# where PROFITs > 1000 and CURRENCY is either Australian Dollar or US Dollar

# Tables used - 
SELECT * from FactInternetSales
SELECT * from DimCurrency

# Query - 

SELECT  fis.ProductKey, fis.CustomerKey, fis.CurrencyKey, fis.SalesTerritoryKey, fis.SalesOrderNumber
		, fis.SalesOrderLineNumber, fis.OrderQuantity, fis.TotalProductCost, fis.SalesAmount
		, (fis.SalesAmount - fis.TotalProductCost) AS PROFIT

		,dp.ProductKey, dp.EnglishProductName, dp.ProductSubcategoryKey
            
		,dps.EnglishProductSubcategoryName, dps.ProductSubcategoryKey, dps.ProductCategoryKey
            
		,dpc.EnglishProductCategoryName, dpc.ProductCategoryKey

		,dc.CurrencyName
            
		,dst.SalesTerritoryCountry
            
		,cst.FirstName, cst.LastName, cst.GeographyKey
            
		,dg.EnglishCountryRegionName
from FactInternetSales as fis
join DimProduct as dp ON dp.ProductKey=fis.ProductKey
join DimProductSubcategory as dps ON dp.ProductSubcategoryKey= dps.ProductSubcategoryKey
join DimProductCategory as dpc ON dpc.ProductCategoryKey= dps.ProductCategoryKey
join DimCurrency as dc ON dc.CurrencyKey=fis.CurrencyKey
join DimSalesTerritory as dst ON dst.SalesTerritoryKey=fis.SalesTerritoryKey
join DimCustomer as cst ON cst.CustomerKey= fis.CustomerKey
join DimGeography as dg ON dg.GeographyKey=cst.GeographyKey

where fis.SalesAmount - fis.TotalProductCost >1000 AND fis.CurrencyKey in (100,6)
order by profit desc

In [ ]:
# Difference betwen a LEFT and INNER JOIN

# Example :
SELECT * from DimCurrency  # 105. This table has 105 Currencies
SELECT * from FactInternetSales  # 60398. This table includes all sales data for each product along with the Currency used for sale.
SELECT count(Distinct Currency) from FactInternetSales  # 06. Only 6 currencies are traded for sale.

# INNER JOIN
SELECT dc.*, fis.ProductKey, fis.OrderQuantity
  From DimCurrency as dc
  Join FactInternetSales as fis ON dc.CurrencyKey = fis.CurrencyKey  # 60398

# LEFT JOIN
SELECT dc.*, fis.ProductKey, fis.OrderQuantity
  From DimCurrency as dc
  Left Join FactInternetSales as fis ON dc.CurrencyKey = fis.CurrencyKey  # 60497. This is the 60398 + 99 (Currencies not there in FIS). 

In [ ]:
! IMPORTANT

QUESTION - What should the DimProduct table store - Subcategory or the Category or Both?  
ANSWER - Subcategory, why? Acc. to Kimball Dimensional Modeling Rule (Industry Standard) - 
"Store only the foreign key for the lowest level of the hierarchy. Higher levels must be derived through relational joins." 
This is actually an OLAP STAR SCHEMA where one central FACT table is connected to multiple Dimension Tables.

QUESTION - Why not Both be kept in the fact table?  
ANSWER - Redundancy. We have a separate table for figuring Category out of Subcategory. We dont need both keys here. We just need one. 

The hierarchy must be - DimProduct → DimProductSubcategory → DimProductCategory. Product references Subcategory and Subcategory references Category.

QUESTION - Does the order of the INNER JOIN above affects the result?   
ANSWER - NO, INNER joins unlike other joins are both COMMUTATIVE (2 × 3 = 3 × 2 ) and ASSOCIATIVE ((2 × 3) × 4 = 2 × (3 × 4). It means SQL Server’s optimizer can reorder INNER JOINs freely.

QUESTION - Can a JOIN happen without a join condition?

## RELATIONSHIP AND CARDINALITY  

What Are “Relationships” in SQL? -- A relationship describes how two tables connect based on a key (usually a primary key to a foreign key).
Example:
DimProductCategory → DimProductSubcategory → DimProduct

What Is “Cardinality”?
Cardinality defines how many rows from one table relate to how many rows in another table.
Cardinality helps you choose correctly between: INNER, LEFT, FULL JOIN

There are three main types: 
a. One - to – One. 
   Each row in Table A relates to exactly one row in Table B and vice‑versa
b. One - to – Many.  
   A single row in Table A can relate to many rows in Table B. But each row in Table B relates back to only one row in Table A.
c. Many - to – Many.
   A record in Table A can relate to many records in Table B. AND a record in Table B can relate to many records in Table A.
   NOTE : Many-to-many relationships always require a bridge table, or you get duplicates × duplicates

In [ ]:
# Example - What is the total sales amount per Product Subcategory?

# QUERY 01 -
SELECT dps.ProductSubcategoryKey, dps.EnglishProductSubcategoryName, sum(fis.SalesAmount)  
    From FactInternetSales as fis 
    left Join DimProduct as dp on fis.ProductKey = dp.ProductKey
    left Join DimProductSubcategory as dps on dp.ProductSubcategoryKey = dps.ProductSubcategoryKey
    group by dps.ProductSubcategoryKey, dps.EnglishProductSubcategoryName
    order by dps.ProductSubcategoryKey


#VERSUS

#QUERY 02 -
WITH ProductSales AS (
    SELECT ProductKey,
           SUM(SalesAmount) AS ProductSalesAmount
    FROM FactInternetSales
    GROUP BY ProductKey
)
SELECT psc.EnglishProductSubcategoryName,
       SUM(ps.ProductSalesAmount) AS TotalSales
    FROM DimProductSubcategory psc
    JOIN DimProduct dp
         ON dp.ProductSubcategoryKey = psc.ProductSubcategoryKey
    LEFT JOIN ProductSales ps
         ON ps.ProductKey = dp.ProductKey
    GROUP BY psc.EnglishProductSubcategoryName;

NOTES:
1. Query 01 is actually finding the Sales of Subcategories that have sales. While Query 02 represent all subcategories even those that dont have sales. 
2. SUBCATEGORY is the keyword and not the SALES.
3. Q01 is going from SALES table to SUBCATEGORIES while Q02 is moving from SUBCATEGORIES to SALES. It means we have to first figure out all 
   subcategories and then go for sales they have. This way we will be able to include the Subcategories which dont have sales as well which is not a 
   possibility with Q01.
4. Moreover, the Q02 aggregates the Sales table first before joining it in order to remove duplicates, then uses a INNER JOIN to keep only SC's in the    DimProduct table (It has Products without SC's as well) and finally taking a Left join to include zero-sales subcategories instead of Inner join.
5. Q02 is Cardinally correct and offers better performance esp. on large fact tables (pre‑aggregation reduces join volume). Also, we are open to join     more dimensions later in this query.

In [ ]:
# Full Outer join

SELECT *
from DimProductCategory as dpc
full outer join DimProductSubcategory as psc ON dpc.ProductCategoryKey = psc.ProductCategoryKey

! IMPORTANT
NOTE: Left outer join is same as Left join

In [ ]:
# Cross join -  returns the Cartesian product of two tables—meaning every row of the first table is paired with every row of the second table.
# NOTE: There is no join condition in a CROSS JOIN.

SELECT *
from DimProductCategory
cross join DimProductSubcategory

# UTILITY
# 1. You use CROSS JOIN to generate all possible pairings.
# 2. Generate date ranges by cross joining a list of numbers.
# 3. Creating test data. You can quickly generate thousands of synthetic rows using CROSS JOINs on small seed tables

In [ ]:
# Equi Join : Any query with equal operator. Similarly Non-Equi join is all using <>,<=,>=

SELECT *
    from DimProductCategory as dpc, DimProductSubcategory as psc
    where dpc.ProductCategoryKey = psc.ProductCategoryKey

# standard joins are preferred then the above way of using equi join. 

# E.g of Non-equi Join
SELECT student.name, record.id, record.city
    FROM student, record
    WHERE student.id < record.id;

In [ ]:
# Self Join

SELECT de.EmployeeKey, de.ParentEmployeeKey, de.FirstName, mgr.FirstName as [Manager's name]
    from DimEmployee as de
    Left join DimEmployee as mgr ON de.ParentEmployeeKey=mgr.EmployeeKey

In [ ]:
# !Important
# What if you do this? 

SELECT de.EmployeeKey, de.FirstName as EmployeeName, dee.FirstName as ManagerName 
FROM DimEmployee as de
join DimEmployee as dee on de.EmployeeKey = dee.ParentEmployeeKey

# SQL BUILT-IN FUNCTIONS
## SQL Server Scalar functions

1. String functions
2. Mathematical
3. Datetime
4. Conditional logic and Comparisons
5. Conversion and Type cast
6. NULL Handling Functions

### STRING FUNCTIONS

1. CONCAT
if any of the 1st/middle/last name is Null then the resut is null
Select FirstName, MiddleName, LastName, FirstName + ' ' + MiddleName + '' + LastName as fullName
from DimCustomer

CONCAT removes Null issue that happened with the above query. CONCAT() treats NULL as empty string
Select FirstName, MiddleName, LastName, CONCAT(FirstName,' ', MiddleName,' ', LastName) as fullName
from DimCustomer

2. CONCAT_WS
CONCAT_WS indicates concatenate with separator
Select FirstName, MiddleName, LastName, CONCAT_WS(' ',FirstName, MiddleName, LastName) as fullName
from DimCustomer

3. SOUNDEX function

0 score means very far away sounds and 4 means the sounds of the words are same. 
DIFFERENCE function tells us this score
select soundex('praveen'),SOUNDEX('praviin'),DIFFERENCE('praveen','praviin')
select soundex('praveen'),SOUNDEX('traveen'),DIFFERENCE('praveen','traveen')

UTILITY
Use SOUNDEX when you need fuzzy matching of names or words
SELECT *
FROM Employees
WHERE SOUNDEX(LastName) = SOUNDEX('Jonson')
Results:
This will match:
<!-- # Johnson
# Jonson
# Jonsen
# Jhnson -->

4. LTRIM, RTRIM, TRIM

LTRIM : removes leading spaces from a string
In the below query STR() pads the input with additional spoaces in the length of 15, which is removed by LTRIM. 
select CustomerKey, FirstName + ' has yearly income of ' + LTRIM(str(YearlyIncome,15,2))
from DimCustomer

The improved query would look like
select CustomerKey, CONCAT(FirstName ,' has yearly income of ',LTRIM(str(YearlyIncome,15,2)))
from DimCustomer

RTRIM : RTRIM() removes trailing (right-side) spaces from a string
SELECT RTRIM('Hello    ')

TRIM() — removes both left and right spaces
select DATALENGTH('     Hello  ')  # Output - 12
select len(Trim('     Hello  '))  # Output - 5

5. LEN, DATALENGTH
For finding the exact length of a string we can use either of 
1. LEN() - It incudes all spaces inside or leading but not the Trailing ones.
2. DATALENGTH() - Includes all spaces but it counts bytes. Therefore if the datatype we are counting is VARCHAR then the no 
   of bytes are same as length but if the datatype is NVARCHAR then the bytes per character assigned is 2 that means the 
   output of datalength(for a nvarchar) needs to be divided by 2 to get exact length.

SELECT len('Hello    ')  # Answer - 5. LEN() function ignores trailing spaces but not the Leading spaces.
SELECT len('     Hello')  # Answer - 10.
SELECT len(RTRIM('Hello    '))  # Answer - 5
SELECT DATALENGTH('Hello    ')  # Answer - 9. If this is a NVARCHAR then the output would be 18 since each character is assigned 2 bytes. 
Therefore we have to divide by 2 to get exact length.

6. LEFT, RIGHT, SUBSTRING
SELECT LEFT('Gurugram', 3);  # Output - Gur
SELECT RIGHT('Gurugram', 4);  # Output - gram
SELECT SUBSTRING('Analytics', 2, 4);  # Output - naly 
SUBSTRING(expression, start, length-Number of characters/bytes to return)
The SUBSTRING function in SQL Server is used to extract a portion of a string. Positions are 1-based
Declare @email varchar(50) = 'mandeep.kumar@gmail.com'
SELECT SUBSTRING(@email, CHARINDEX('@', @email) + 1, 10) AS DomainName   # Output - gmail.com

7. CHARINDEX()
Finds a substring's position (case‑insensitive by default)
NOTE : Position (1-based index) where the substring first occurs. If the substring is not found, it returns 0.
CHARINDEX ( expressionToFind , expressionToSearch [ , start_location ] )
SELECT CHARINDEX('dee', 'Mandeep');  # Output - 4
SELECT CHARINDEX('dee', 'Mandeep',5);  # Output - 0

8. PATINDEX
<!-- # Pattern‑based search using wildcards (%, _) -->
SELECT PATINDEX('%eep', 'Mandeep');  # Output - 5

9. REPLACE
<!-- # Replaces ALL occurrences of a substring -->
SELECT REPLACE('SQL Server', 'SQL', 'Azure')  # Output - Azure Server

10. UPPER, LOWER
SELECT UPPER('mandeep');  # Output - MANDEEP
SELECT LOWER('ANALYST');  # Output - analyst

11. STRING_AGG
STRING_AGG : is an aggregate function that takes all expressions from rows and concatenates them into a single string.
SELECT string_agg(firstName,'--') as CSV
from DimCustomer  # Output - Aaron--Aaron--Aaron--.....
NOTE : if the no of rows are many then the above query will throw EXCEED LIMIT OF 8000 BYTES error. This is because it outputs the same datatype.

12. FORMAT
Format any value as string
SELECT FORMAT(12345, '##,##0')   # Output - 12,345

13. STRING_SPLIT
Split a delimited string into rows and returns a table with a column named VALUE. 
For preserving the order, use the optional argument as True or False to enable or diable order
STRING_SPLIT ( string , separator [,enable_ordinal ] )
SELECT value FROM STRING_SPLIT('A,B,C', ',')   # Output - Value (as a column) - A, B, C
SELECT value, ordinal FROM STRING_SPLIT('A,B,C', ',', 1); # Output - Two columns- Value with values - A, B, C and Column ordinal with values 1, 2, 3.

14. List Aggregate or STRING_AGG
STRING_AGG(expression, delimiter) with optional WITHIN GROUP (ORDER BY ...) supported through an ORDER BY in WITHIN GROUP.
What it does? - It concatenate multiple row values into a single, delimited string—typically within each group.

-- Example 01
SELECT Dept,
       STRING_AGG(Emp, ', ') AS Employees
FROM dbo.Staff
GROUP BY Dept;

-- Example 02
SELECT Dept,
       STRING_AGG(Emp, ', ') WITHIN GROUP (ORDER BY Emp) AS Employees
FROM dbo.Staff
GROUP BY Dept;

### MATHEMATICAL FUNCTIONS
ABS(-5)          # Output - 5
CEILING(4.2)     # Output - 5
FLOOR(4.9)       # Output - 4
ROUND(123.456, 2) # Output - 123.46
POWER(2, 3)      # Output - 8
SQRT(16)         # Output - 4
RAND()           # Output - random number
SIGN(-10)        # Output - -1 


1. FLOOR, CEILING, ROUND
SELECT ExtendedAmount
	, floor(ExtendedAmount) as 'Floor' 
	, CEILING(ExtendedAmount) as 'Ceiling'
	, round(ExtendedAmount,0) AS 'Round0'
	, round(ExtendedAmount,1) AS 'Round1'
	, round(ExtendedAmount,2) AS 'Round2' 	
from FactInternetSales

NOTE: Round 0 truncate and outputs still in the same decimal datatype and not integer. 

### DATETIME functions

SELECT 	SYSDATETIME ( )  #  OUTPUT : 2026-02-02 10:11:13.5529888. Outputs the clients datetime
SELECT SYSDATETIMEOFFSET ( )  # OUTPUT : 2026-02-02 10:11:57.1024740 +05:30. Includes timeoffset as well
SELECT 	SYSUTCDATETIME ( )  # OUTPUT : 2026-02-02 04:42:17.3983016. Returns the UTC time of this system
NOTE: The above 3 outputs higher precision values and getdate() Outputs lower precision
SELECT GETDATE ( )  # OUTPUT : 2026-02-02 10:12:40.267. 

1. DATE_BUCKET() - It is used to group date and time values into fixed intervals (buckets). It calculates the start of each bucket 
based on the specified interval, making it useful for time-series analysis or aggregating data over specific period. 
DATE_BUCKET (datepart, number, date [, origin]). NOTE: origin (optional): The starting point for bucketing. Defaults to 1900-01-01 00:00:00.000.
SELECT DATE_BUCKET(DAY, 10, getdate())  # OUTPUT : 2026-01-30 00:00:00.000
SELECT DATE_BUCKET(Year, 10, getdate()) # OUTPUT : 2020-01-01 00:00:00.000
SELECT DATE_BUCKET(Month, 10, getdate()) # OUTPUT : 2025-11-01 00:00:00.000. 

UTILITIES: 
Group data into weekly buckets starting from the default origin:
DECLARE @date DATETIME2 = '2023-10-15 14:30:00' 
SELECT DATE_BUCKET(WEEK, 1, @date) AS WeeklyBucket
NOTE : This is how we create a variable. Here the name of the it is date and the DATETIME2 signifies the type of the variable. 
E.g
DECLARE @age INT = 30;
DECLARE @name VARCHAR(50) = 'Mandeep'
DECLARE @city NVARCHAR(100) = 'Gurugram' . 
!Important : NVARCHAR stores Unicode characters (UTF‑16) and the preceeding N tells SQL Server that the string is Unicode.
If we don’t put the N prefix, SQL Server treats the string as a non‑Unicode VARCHAR literal, which means: it may lose data and 
characters outside ASCII might break or become question marks. 


2. Quick comparisons: DATE vs DATETIME vs DATETIME2 vs DATETIMEOFFSET
DATE → Only date, no time.
DATETIME → Older type; lower precision (~3.33 ms), limited range, sometimes causes rounding.
DATETIME2 → Best general-purpose timestamp: higher precision, larger range, smaller storage (at some precisions).
DATETIMEOFFSET → Stores date-time with time zone offset; use this if you care about UTC or offsets.

3. DATEPART()
Returns a numeric part of the date.
SELECT DATEPART(month, '2025-02-10');  # OUTPUT : 2

4. DATENAME()
Returns the textual name of part of a date.
SELECT DATENAME(weekday, '2025-02-10');  # OUTPUT : Monday

5. DAY(), MONTH(), YEAR()
SELECT YEAR('2025-02-10');
SELECT MONTH('2025-02-10');
SELECT DAY('2025-02-10');

Add or Subtract Dates : DATEADD, DATEDIFF
6. DATEADD
DATEADD (datepart, number, date). NOTE: Number could be +/-
E.g : 
SELECT DATEADD(year, 1, GETDATE()) AS DateAdd   # OUTPUT : 2027-02-02 10:52:30.097. Adding One Year to a Date
SELECT DATEADD(month, 2, GETDATE()) AS DateAdd  # OUTPUT : 2026-04-02 10:52:55.940. Adding Two Months to a Date
NOTE: Handling Month Overflow. 
If you add a no. of months to a date and the resulting day does not exist in the return month, the DATEADD function 
returns the last day of the return month. For example, adding four months to '2019-05-31' results in '2019-09-30' because September has only 30 days.

7. DATEDIFF
DATEDIFF(datepart, startdate, enddate) 
DATEDIFF function calculates the difference between two dates. It returns the difference as an Integer value
SELECT DATEDIFF(year, '2017-08-25', '2021-08-25') AS DateDiff # OUTPUT : 4

8. Format Dates : FORMAT()
SELECT FORMAT(GETDATE(), 'yyyy-MM-dd')

9. CAST and CONVERT
SELECT CAST(GETDATE() AS DATE) # OUTPUT : 2026-02-02. 
SELECT CONVERT(date, SYSDATETIME()) # OUTPUT : 2026-02-02. 

10. Validate Dates
SELECT ISDATE('2025-02-30');  # 0 (invalid)

11. Start/End of Periods
Returns the last day of the month that is a specified number of months before or after a given start date
EOMONTH(start_date [, month_to_add])
SELECT EOMONTH('2024-02-15') AS LastDayOfMonth # OUTPUT : 2024-02-29
SELECT EOMONTH(getdate(),1) # OUTPUT : 2026-03-31
SELECT EOMONTH(getdate(),-2) # OUTPUT : 2025-12-31

### CONDITIONAL LOGIC AND COMPARISONS

1. IIF()
IIF is a shortcut for CASE WHEN
IIF(condition, true_value, false_value)
SELECT IIF(10 > 5, 'Bigger', 'Smaller')  # OUTPUT : Bigger
SELECT SalesAmount
      , TotalProductCost
      , IIF((SalesAmount - TotalProductCost) > 1000, 'High Value', IIF((SalesAmount - TotalProductCost) > 500,'Mid value', 'low value')) as 'Trans_type'
FROM FactInternetSales  

2. GREATEST()
Returns the largest value from a list
SELECT GREATEST(10, 25, 7) # OUTPUT : 25

3. LEAST()
Returns the smallest (least) value from the list.
SELECT LEAST(10, 25, 7)  # OUTPUT : 7

### CONVERSION / TYPE CAST FUNCTIONS

1. CAST(value AS INT)
2. CONVERT(VARCHAR(10), GETDATE(), 120)
3. PARSE('12,345' AS INT USING 'en-US')
4. TRY_CAST('abc' AS INT)  -- returns NULL instead of error

### NULL HANDLING FUNCTIONS

1. NULLIF(x, y)      -- returns NULL if x = y
2. COALESCE(a, b, c) -- returns first non-null value
3. ISNULL(column, 'default')


## Window Functions

Window functions let you perform calculations across a set of rows that are related to the current row—without 
collapsing rows like GROUP BY does. They help you calculate things like:

Running totals
Row numbers
Ranking
Moving averages
Percentiles
Comparisons within groups


### 1. The OVER Clause
Every window function needs an OVER clause. It defines the “window” — the set of rows over which the function operates.
E.g:
SELECT
    SalesOrderID,
    LineTotal,
    SUM(LineTotal) OVER () AS total_sales_across_whole_table
FROM Sales.SalesOrderDetail;

NOTE : 
SUM(LineTotal) is the window function and OVER() defines the window.
Since nothing is inside OVER(), the window = all rows of the table

### 2. PARTITION BY
PARTITION BY is like GROUP BY for window functions, but with one key difference: Window functions do NOT reduce rows.
They keep all rows but calculate values within each partition thereby allowing us to do calculations within groups.
E.g:
SELECT 
    SalesOrderID,
    LineTotal,
    SUM(LineTotal) OVER (partition by SalesOrderID) AS total_sales_across_whole_table
FROM Sales.SalesOrderDetail

NOTE: 
Each row displays: One SalesOrderID and its LineTotal
🚀 And the total for each SalesOrderID, repeated on every row of that SalesOrderID. 
This is why window functions are so powerful — they add information without removing detail.

### What Are Ranking Functions?
Ranking functions assign a rank to each row, based on the order you define in an OVER() clause

### 3. ROW_NUMBER()
Row numbers always give a unique sequential number within each partition.
No ties → every row gets a unique number
If two rows have same values, SQL still breaks ties arbitrarily.
Sales: 10, 10, 9, 8
ROW_NUMBER(): 1, 2, 3, 4

### 4. RANK()
RANK assigns the same rank for ties, and then skips numbers after a tie.
Values: 10, 10, 9, 8
RANK(): 1, 1, 3, 4
Explanation: Two rows tie for 1st → both ranked 1. Next rank becomes 3 (skips 2)

### 5. DENSE_RANK()
Similar to RANK, but does not skip numbers after ties.
Values: 10, 10, 9, 8
DENSE_RANK(): 1, 1, 2, 3

### 6. NTILE(n)
NTILE(N) divides rows into N equal groups within each partition. This bucketing is done inside a window when used with PARTITION BY clause.
Use cases:
Split customers into top/mid/low groups
Divide employees into performance quartiles
Divide students into grade bands
Divide products into sales buckets

E.g:
SELECT
    YEAR(soh.OrderDate) AS SalesYear,
    soh.CurrencyRateID,
    SUM(sod.LineTotal) AS TotalOrder,
    NTILE(3) OVER (
        PARTITION BY YEAR(soh.OrderDate)
        ORDER BY SUM(sod.LineTotal) DESC
    ) AS TertileGroup
FROM Sales.SalesOrderHeader soh
JOIN Sales.SalesOrderDetail sod 
    ON sod.SalesOrderID = soh.SalesOrderID
GROUP BY YEAR(soh.OrderDate), soh.CurrencyRateID
ORDER BY SalesYear ASC;

NOTE: Three buckets are created inside each Year. The buckets are created taking order of Sum of LineTotal.
If 2011 30 rows then NTILE(3) splits them almost equally:
Bucket 1 → 10 rows
Bucket 2 → 10 rows
Bucket 3 → 10 rows

NOTE: 
1. NTILE with PARTITION BY will create buckets inside each window.
2. NTILE without P_BY will take all rows as one window and then create buckets.


### 7.  LAG()
Accesses data from a previous row based on a given offset.
Use case: Compare current row values with previous row values.
LAG(column, offset, default_value) OVER (ORDER BY col)
offset = how many rows back (default = 1)
default_value = what to return if no previous row exists

### 8. LEAD()
Accesses data from a subsequent row based on a given offset
Use case: Compare current row values with next row values.
LEAD(column, offset, default_value) OVER (ORDER BY col)

!IMPORTANT: Lead and Lag functions let you look ahead or look back at another row without doing a SELF-JOIN.

E.g:
SELECT
    YEAR(soh.OrderDate) AS SalesYear,
    soh.SalesOrderID,
    sod.LineTotal,

    LAG(sod.LineTotal) OVER (
        PARTITION BY YEAR(soh.OrderDate)
        ORDER BY soh.OrderDate
    ) AS PrevOrderAmount,

    LEAD(sod.LineTotal) OVER (
        PARTITION BY YEAR(soh.OrderDate)
        ORDER BY soh.OrderDate
    ) AS NextOrderAmount

FROM Sales.SalesOrderHeader soh
JOIN Sales.SalesOrderDetail sod 
     ON sod.SalesOrderID = soh.SalesOrderID

NOTE:
1. Window Functions Are Calculated AFTER the WHERE Clause and CANNOT be used directly in the WHERE clause. The Correct way is to use a subquery or CTE.
E.g:
WITH cte AS (
   SELECT *,
          ROW_NUMBER() OVER (ORDER BY OrderDate) AS rn
   FROM Orders
)
SELECT *
FROM cte
WHERE rn = 1;

## TOP N CLAUSE

### 1. TOP N Clause 
SELECT TOP 5* FROM Sales ORDER BY Amount DESC;
NOTE : If there are ties (same value), SQL does NOT automatically include them.
Example: Sales values: 100, 95, 95, 95, 90, 85
TOP 3 ORDER BY Sales DESC gives: 100, 95, 95
But it excludes the 3rd 95 if it’s not in that row count. This is where WITH TIES comes in.

### 2. TOP N WITH TIES
SELECT TOP (3) WITH TIES * FROM Sales ORDER BY Amount DESC  
Return the top 3 rows plus any tied rows matching the last row's ORDER BY value
Example: Sales values: 100, 95, 95, 95, 90, 85
TOP 3 ORDER BY Sales DESC gives: 100, 95, 95, 95 # RESULT HAVE 4 ROWS!

NOTE: 
ROW_NUMBER() is similar to TOP N (no ties).
RANK() is similar to TOP N WITH TIES

## SET OPERATORS

Set operators combine rows from two result sets vertically (like stacking tables on top of each other).
They follow mathematical "SET" operations
SQL Server supports four major set operators:
1. UNION : A ∪ B (distinct rows from both)
2. UNION ALL : All rows from A + all from B
3. INTERSECT : A ∩ B (common rows)
4. EXCEPT : A − B (rows in A but not in B)

NOTES:
1. Rule 1 — Both queries must have same structure. Every SET operator requires:
✔ Same number of columns
✔ Same data types (compatible)
✔ Same column order

2. Comparison with JOINS
Joins = combine columns (horizontal)
Set operators = combine rows (vertical)

# SUBQUERIES
1. Single Row Sub Queries  
2. Multi Row Sub Queries  
3. Built-in function  
4. Nested Sub Queries  
5. Correlated Sub Queries  
6. Derived tables  
7. Recursive queries  
8. Dynamic Queries and Pivots  
9. Common Table Expression


What is a Subquery? 
Used when a query needs the result of another query. A subquery is a SELECT statement inside:
1. WHERE
2. SELECT (E.g: SELECT EmpID, Salary, (SELECT AVG(Salary) FROM Employees) AS AvgCompanySalary FROM Employees)
3. FROM
4. HAVING
5. INSERT / UPDATE


## Single‑Row Subqueries
These return one row, one value.

## Multi‑Row Subqueries
Return multiple rows.
Used with:
IN, NOT IN  (SELECT * FROM Employees WHERE DeptID IN (SELECT DeptID FROM Departments WHERE Location = 'Delhi')) 
ANY (SELECT * FROM Employees WHERE Salary > ANY (SELECT Salary FROM Interns)) # Finds salary > minimum intern salary.
ALL (SELECT * FROM Employees WHERE Salary > ANY (SELECT Salary FROM Interns)) # Finds salary > highest intern salary.

NOTE: SQL SERVER suports both SOME and ANY which are synonyms. They return TRUE if the comparison is satisfied for at least one value in the set. They differ from ALL, which requires the condition to be true for every value in the set

E.g - Are there any customers that never ordered
SELECT CustomerKey
FROM DimCustomer
WHERE customerkey NOT IN (SELECT CustomerKey FROM FactInternetSales)

## Subqueries Using Built‑in Functions
Helpful when using:COUNT, SUM, AVG, MIN, MAX
E.g: WHERE Salary > (SELECT AVG(Salary) FROM Employee)

## Nested SubQueries
E.g - Find All employees having salaries > average of IT department employee salaries.
SELECT EmpName FROM Employees
WHERE Salary > (
      SELECT AVG(Salary) FROM Employees
      WHERE DeptID = (
             SELECT DeptID FROM Departments
             WHERE DeptName = 'IT'
      )
)

## Correlated Subqueries
A correlated subquery depends on the outer query
E.g: Compare each employee’s salary to the average salary of their own department
SELECT e1.EmpName, e1.Salary FROM Employees e1
WHERE e1.Salary >
      (SELECT AVG(Salary) FROM Employees e2
       WHERE e2.DeptID = e1.DeptID);
NOTE:
✔ Subquery runs once per row of outer query
✔ Slower but extremely powerful
✔ Often replaced by window functions

## Derived Tables (Subqueries in FROM)
A subquery acting like a temporary table.
E.g:
SELECT DeptID, AvgSal
FROM (
     SELECT DeptID, AVG(Salary) AS AvgSal
     FROM Employees
     GROUP BY DeptID
) AS t
WHERE AvgSal > 50000;

## Common Table Expressions (CTEs)
A CTE is a named derived table that is readable and reusable.
Why CTE is preferred over derived tables?
✔ More readable
✔ Can be referenced multiple times
✔ Allows recursive queries
✔ Helps break complex queries into parts


## Recursive CTEs
Used for:
Hierarchy (manager → employee)
Parent/child structures
Tree traversal
Organizational charts

## Dynamic SQL
SQL constructed as a string and executed with EXEC() or sp_executesql.
E.g
DECLARE @sql NVARCHAR(MAX)
SET @sql = 'SELECT * FROM Employees WHERE DeptID = 5'
EXEC(@sql)

Used when:
✔ Table name is dynamic
✔ Column name is dynamic
✔ Pivoting dynamically
✔ Search filters vary

## Pivot and Unpivot
PIVOT transforms rows → columns and UNPIVOT do columns → rows
E.g: Convert department salary summary into columns:
SELECT *
FROM ( SELECT DeptName, Salary FROM Employees
    ) src
PIVOT (
    AVG(Salary) FOR DeptName IN ([IT], [HR], [Finance])
) AS p;

## SUBQUERY VS JOINS

E.g - Writing multivalued Query using both Subquery and join and examine the time taken by each

QUERY01
SELECT SalesOrderNumber, SalesAmount
FROM FactInternetSales
WHERE CustomerKey IN (SELECT CustomerKey FROM DimCustomer WHERE GeographyKey=5)

QUERY02
SELECT fis.SalesOrderNumber, fis.SalesAmount
FROM FactInternetSales AS fis
JOIN DimCustomer AS dc ON fis.CustomerKey=dc.CustomerKey
WHERE GeographyKey=5

1. HOW TO FIND THE TIME TAKEN BY BOTH QUERY - Use CNTRL+L in SSMS or the button - Display Estimated Execution Plan. This is display the plan ssms took to execute both th queries with time consumed between both the queries.
2. TIME TAKEN - Both took the same times
3. Moreover, if u want a detailed analysis if the execution plan use the below -
SET STATISTICS IO, TIME ON
Query 01
Query 02
SET STATISTICS IO, TIME OFF

Notice these 2 - 
3.1 Logical Reads - (Number of data pages read from the buffer cache (not disk) and represent the amount of work SQL Server did to read data) and 
3.2 CPU time - Actual processing time spent by the CPU

4. What shall we prefer - A JOIN or SUBQUERY.
BEST PRACTICES
When GOAL = Get columns from both tables THEN USE JOIN
When GOAL = Simple filtering
         AND uniqueness guaranteed of the column used for filtering THEN USE JOIN. FOr e.g in the above eg IN would be preferred
         if CustomerKey is Unique and not duplicated in the Dimcustomer table. 
When GOAL = Only filter based on matching
          AND Avoid row duplication
          AND Avoid NULL issues
          THEN USE EXISTS. It is Faster for filtering, avoids duplicates
E.g -
SELECT fis.SalesOrderNumber, fis.SalesAmount
FROM FactInternetSales fis
WHERE EXISTS (
    SELECT 1
    FROM DimCustomer dc
    WHERE dc.CustomerKey = fis.CustomerKey
      AND dc.GeographyKey = 5);

E.g -
select count(distinct(CustomerID)) from sales.Customer --19820
select count(distinct(CustomerID)) from sales.SalesOrderHeader --19119

select sc.CustomerID from sales.customer as sc
where exists (select * from sales.SalesOrderHeader as soh where soh.CustomerID=sc.CustomerID) --19119

## CORRELATED QUERY VS WINDOW FUNCTION

E.g 
1. QUERY 01 - WITH CORRELATED SUBQUERY

SET STATISTICS IO, TIME ON;
with cte as (
SELECT soh01.SalesOrderID, soh01.SalesPersonID
FROM sales.SalesOrderHeader AS soh01
WHERE soh01.SalesOrderID=
	(SELECT Max(soh02.SalesOrderID) 
		FROM sales.SalesOrderHeader AS soh02
		WHERE soh01.SalesPersonID=soh02.SalesPersonID)
)
select * from cte where SalesPersonID=279
SET STATISTICS IO, TIME OFF

2. QUERY 02 - WITH WINDOW FUNCTION

SET STATISTICS IO, TIME ON;
with ranked_cte as (
	select salesPersonID
			, SalesOrderID
			, ROW_NUMBER() over (Partition by SalesPersonID order by SalesOrderID DESC) as rn
		from sales.SalesOrderHeader
	)
select * from ranked_cte where SalesPersonID=279 
SET STATISTICS IO, TIME OFF

NOTE:
1. The time taken for above queries seem to favor the correlated query apprach but thats a misnomer as the table is relatively small.
2. On larger tables Window function is always preferred over correlated queries.

## Example of CTE

In [ ]:
WITH Sales_CTE (SalesPersonID, SalesYear, TotalSales)
AS
# Define the first CTE query.
(
    SELECT SalesPersonID,
           YEAR(OrderDate) AS SalesYear,
           SUM(TotalDue) AS TotalSales
    FROM Sales.SalesOrderHeader
    WHERE SalesPersonID IS NOT NULL
    GROUP BY SalesPersonID, YEAR(OrderDate)
)
, # Use a comma to separate multiple CTE definitions.
# Define the second CTE query, which returns sales quota data by year for each sales person.

Sales_Quota_CTE (BusinessEntityID, SalesQuotaYear, SalesQuota)
AS
(
    SELECT BusinessEntityID,
           YEAR(QuotaDate) AS SalesQuotaYear,
           SUM(SalesQuota) AS SalesQuota
    FROM Sales.SalesPersonQuotaHistory
    GROUP BY BusinessEntityID, YEAR(QuotaDate)
)
# Define the outer query by referencing columns from both CTEs.

SELECT SalesPersonID,
       SalesYear,
       FORMAT(TotalSales, 'C', 'en-us') AS TotalSales,
       SalesQuotaYear,
       FORMAT(SalesQuota, 'C', 'en-us') AS SalesQuota,
       FORMAT(TotalSales - SalesQuota, 'C', 'en-us') AS Amt_Above_or_Below_Quota
FROM Sales_CTE
     INNER JOIN Sales_Quota_CTE
         ON Sales_Quota_CTE.BusinessEntityID = Sales_CTE.SalesPersonID
            AND Sales_CTE.SalesYear = Sales_Quota_CTE.SalesQuotaYear
ORDER BY SalesPersonID, SalesYear;

## Example of RECURSIVE QUERIES

In [ ]:
# Creating an Employee table for recursive use case

CREATE TABLE dbo.MyEmployees
(
    EmployeeID SMALLINT NOT NULL,
    FirstName NVARCHAR (30) NOT NULL,
    LastName NVARCHAR (40) NOT NULL,
    Title NVARCHAR (50) NOT NULL,
    DeptID SMALLINT NOT NULL,
    ManagerID SMALLINT NULL,
    CONSTRAINT PK_EmployeeID PRIMARY KEY CLUSTERED (EmployeeID ASC),
    CONSTRAINT FK_MyEmployees_ManagerID_EmployeeID FOREIGN KEY (ManagerID) REFERENCES dbo.MyEmployees (EmployeeID)
);

# Populate the table with values.
INSERT INTO dbo.MyEmployees VALUES
(1, N'Ken', N'Sánchez', N'Chief Executive Officer', 16, NULL),
(273, N'Brian', N'Welcker', N'Vice President of Sales', 3, 1),
(274, N'Stephen', N'Jiang', N'North American Sales Manager', 3, 273),
(275, N'Michael', N'Blythe', N'Sales Representative', 3, 274),
(276, N'Linda', N'Mitchell', N'Sales Representative', 3, 274),
(285, N'Syed', N'Abbas', N'Pacific Sales Manager', 3, 273),
(286, N'Lynn', N'Tsoflias', N'Sales Representative', 3, 285),
(16, N'David', N'Bradley', N'Marketing Manager', 4, 273),
(23, N'Mary', N'Gibson', N'Marketing Specialist', 4, 16);


# Recursive query
# Write a query that walks the org chart from the top down and assigns a level to each employee, letting you view or further analyze the hierarchy.

With DirectReports (ManagerID, EmployeeID, Title, EmployeeLevel ) AS
(
	SELECT ManagerID, EmployeeID, Title, 0 AS EmployeeLevel
	FROM dbo.MyEmployees
	WHERE ManagerID IS NULL  # ANCHOR QUERY (BASE CASE). This will only run once. 
	UNION ALL
	SELECT e.ManagerID, e.EmployeeID, e.Title, EmployeeLevel + 1
	FROM dbo.MyEmployees AS e
	JOIN DirectReports AS d
		ON e.ManagerID=d.employeeID -- RECURSIVE PART
)
select ManagerID, EmployeeID, Title, EmployeeLevel
from DirectReports
ORDER BY ManagerID

# A recursive CTE consists of three elements:
# 1. Invocation of the routine.
## The first invocation of the recursive CTE consists of one or more CTE query definitions joined by UNION ALL, UNION, EXCEPT, or INTERSECT operators. 
## Because these query definitions form the base result set of the CTE structure, they're referred to as anchor members.
## CTE query definitions are considered anchor members unless they reference the CTE itself. All anchor-member query definitions must be positioned
## before the first recursive member definition, and a UNION ALL operator must be used to join the last anchor member with the first recursive member.

# 2. Recursive invocation of the routine.
## The recursive invocation includes one or more CTE query definitions joined by UNION ALL operators that reference the CTE itself

# 3. Termination check.
## The termination check is implicit; recursion stops when no rows are returned from the previous invocation

# !IMPORTANT
# The key idea (how SQL Server actually runs it)
# A recursive CTE is evaluated in phases using two internal, conceptual tables:
## W (working set): the rows found in the current iteration.
## R (result set): the accumulated rows from all iterations so far.

### Algorithm (breadth‑first, level‑by‑level):
# 1. Run the anchor member (the top SELECT) → put those rows into W and R.
# 2. Loop:
# 2.1 Run the recursive member, but the reference to the CTE (DirectReports) means “use the rows currently in W”.
# 2.2 The rows produced become the next W (call it W’).
# 2.3 Append W’ to R.
# 2.4 Replace W with W’.
# 3. Stop when the recursive member returns no rows (i.e., W becomes empty).
# 4. The final SELECT FROM DirectReports returns R (everything found).

# NOTE: The self-reference is not circular in one shot — it’s level-by-level
# Each iteration uses only the rows from the previous iteration.

In [ ]:
# OPTIONAL : An Alternative of recursive for the above example. 
# If you know the maximum depth (e.g., at most 3 levels), you can write a series of self-joins.
# Use when: shallow hierarchies, need simple flat output quickly.
SELECT
    L0.EmployeeID AS L0_ID, L0.Title AS L0_Title,
    L1.EmployeeID AS L1_ID, L1.Title AS L1_Title,
    L2.EmployeeID AS L2_ID, L2.Title AS L2_Title
FROM dbo.MyEmployees AS L0
LEFT JOIN dbo.MyEmployees AS L1 ON L1.ManagerID = L0.EmployeeID
LEFT JOIN dbo.MyEmployees AS L2 ON L2.ManagerID = L1.EmployeeID
WHERE L0.ManagerID IS NULL;

# the flow of the above query would be such that after joinin L0 and L1, SQL filters the reulst for WHERE statemtn to optimise and then joins with L2. 

## Example of PIVOT

In [ ]:
SELECT SalesTerritoryKey, SUM(SalesAmount) as SumOfSales from FactInternetSales
group by SalesTerritoryKey
order by SalesTerritoryKey

SELECT 'SumOfSales' AS SalesTerritory, [1],[2],[3],[4],[5],[6],[7],[8],[9],[10]
FROM (SELECT SalesTerritoryKey, SalesAmount FROM FactInternetSales
) AS sourceTable 
PIVOT ( SUM(SalesAmount) FOR SalesTerritoryKey IN ([1],[2],[3],[4],[5],[6],[7],[8],[9],[10]) )
AS TODO


With cte as (
SELECT 'SumOfSales' AS SalesTerritory, [Australia],[United Kingdom]
FROM (SELECT fis.SalesAmount, dst.SalesTerritoryRegion
		FROM FactInternetSales as fis 
		JOIN DimSalesTerritory as dst
		ON fis.SalesTerritoryKey=dst.SalesTerritoryKey
		where dst.SalesTerritoryRegion in ('Australia','United Kingdom')
) AS sourceTable 
PIVOT ( SUM(SalesAmount) FOR SalesTerritoryRegion IN ([Australia],[United Kingdom]) )
AS PivotTable
)
select SalesTerritory, format([Australia], 'C', 'en-in') AS Australia,format([United Kingdom] , 'C', 'en-in') AS UK 
from cte

# VIEWS
A view is a virtual table whose contents are defined by a query. The data come from tables referenced in the view and are produced dynamically
Unless indexed, a view does not exist as a stored set of data values in a database. 

NOTE:
1. We cannot use ORDER BY inside a CREATE VIEW statement without TOP() in the select.
2. This is because a VIEW is unordered set of rows just like a logical table.
3. ORDER BY otherwise will bake the order inside the view which is against this logical table.

## SCHEMABINDING and ENCRYPTION

### SCHEMABINDING
SCHEMABINDING is an option that binds a view or function to the underlying table schema.
What SCHEMABINDING Does:
1.1 Prevents dropping the underlying table.
1.2 Prevents modifying columns used by the view/function (e.g., data type change, rename).
1.3 Makes the view/function more stable and reliable.
1.4 Required for some features like: Indexed views, Deterministic functions

In [ ]:
# Create a base table
Create Table t1 (ID INT,FN VARCHAR(50))

# Create a view with SCHEMABINDING
Create or alter View v1 with SCHEMABINDING 
as
select ID, FN from dbo.t1

# Check the definition of the view v1 from the system catalog.
# NOTE: sys.sql_modules is a system view that stores the SQL text (definition) for modules (views, UDFs, stored procedures, functions, triggers).
# If the object has a body of SQL text, it will appear in sys.sql_modules.
select definition from sys.sql_modules
where object_id = OBJECT_ID('dbo.v1')

### ENCRYPTION 

SQL Server allows encrypting:
1. stored procedure definitions
2. function definitions
3. view definitions
4. triggers
This is done using the WITH ENCRYPTION clause. 

What happens in the backend : It Hides the T‑SQL code from:

1. SSMS “View Definition”
2. sys.sql_modules
3. sp_helptext

Thereby, Protects intellectual property and sensitive logic.

What ENCRYPTION Does NOT Do:
1. Does NOT encrypt the data in tables.
2. Does NOT provide strong security—can be cracked by third‑party tools.

In [ ]:
# Create a view with SCHEMABINDING and ENCRYPTION 
Create or alter View v2
with schemabinding, Encryption
as
select ID, FN from dbo.t1;

# Check the definition of the view v1 from the system catalog.
select definition from sys.sql_modules
where object_id = OBJECT_ID('dbo.v2')
# OUTPUT - Definition is coming as NULL. 

# USER-DEFINED FUNCTIONS

What are UDFs ?
1. Are reusable object that returns a value or table.
2. UDFs are similar to functions in programming languages—they take input parameters and return output
3. But cannot modify database state (e.g., no INSERT/UPDATE/DELETE).

Types :
1. Scalar Functions (Returns one single value)
2. Inline Table‑Valued Function (Returns a TABLE but written as a single SELECT, there's no function body)
3. Multi‑statement Table‑Valued Function (Returns a TABLE, but you can use multiple statements)

What they cannot do:
1. Modify data (INSERT, UPDATE, DELETE, MERGE)
2. Create or alter tables
3. Use TRY...CATCH
4. Use RAISERROR or THROW
5. Call stored procedures
6. Use non-deterministic functions like NEWID() or GETDATE() (unless schema-bound)
7. Execute dynamic SQL (EXEC, sp_executesql)

How to drop a function?
Drop function function_Name

How to check UDFs in the DB?
GO to index of the DB and under 'Programmability' we can find all UDFs, SPs.

## SCALAR UDF

In [ ]:
# Definition of scalar UDF
# EXAMPLE 01
CREATE OR ALTER FUNCTION GiveMeSquare (@x1 float)
RETURNS float
AS BEGIN
	RETURN (@x1 * @x1)
END;

# Calling the UDF
select dbo.GiveMeSquare(3)

select SalesAmount, dbo.GiveMeSquare(salesAmount)
from FactInternetSales

# Dropping the function
Drop function GiveMeSquare

----------------------------------
# EXAMPLE 02 - ADVENTUREWORKS2025
CREATE FUNCTION dbo.ufnGetInventoryStock (@ProductID INT) # Returns the stock level for the product.
RETURNS INT
AS
BEGIN
    DECLARE @ret AS INT;

    SELECT @ret = ISNULL(SUM(p.Quantity),0)
    FROM Production.ProductInventory AS p
    WHERE p.ProductID = @ProductID
          AND p.LocationID = '6';
    # IF (@ret )
    #     SET @ret = 0;
    RETURN @ret;
END

# Calling the UDF
SELECT ProductModelID,
       Name,
       dbo.ufnGetInventoryStock(ProductID) AS CurrentSupply
FROM Production.Product
WHERE ProductModelID BETWEEN 75 AND 80;

In [ ]:
# <!-- NOTE:  -->
# <!-- EXAMPLE 01 -->
CREATE OR ALTER FUNCTION somefunction (@a varchar, @b varchar)
RETURNS int
AS
BEGIN 
RETURN (len(@a) + len(@b))
END;

select dbo.somefunction('string','string')   # The Output will be 2. Why? In T‑SQL, when you declare a varchar without a length, 
# it defaults to varchar(1) for variables and routine parameters. That means both inputs 'string' get truncated to a single character ('s')

# The correct way would be :
CREATE OR ALTER FUNCTION somefunction (@a varchar(10), @b varchar(10))  # Always specify length. The LEN function trims the trailing spaces anyway. 
RETURNS int
AS
BEGIN 
RETURN (len(@a) + len(@b))
END;

---------------------------------------------------
# <!-- EXAMPLE 02 -->
Create or Alter function CheckCustomer(@custKey int)
RETURNS BIT
AS BEGIN
	Declare @result BIT
	IF EXISTS( select 1 from DimCustomer
				WHERE CustomerKey=@custKey )
		set @result = 1
	ELSE
		set @result = 0
	return @result
END;

select dbo.CheckCustomer(CustomerKey) from DimCustomer where CustomerKey = 1100  
# -- Only returns 1 when found and return nothing when CustomerKey is not found, why? eEcause it doesnot call the fucntion if the 
# -- the customerKey is not found. it just return empty from the outer query itself. 
SELECT dbo.CheckCustomer(1100);   -- Always returns 1 or 0
# But this call will return either 0 or 1. 

## INLINE TABLE-VALUED FUNCTION
NOTE : These are sigle line functions without BEGIN and the entire logic is in the RETURN statement itself

In [ ]:
# EXAMPLE 01
create or alter function sales.GetLastOrdersForCustomers(@CustomerID int, @NumberOfOrders int)
Returns TABLE
As
Return (select top(@NumberOfOrders) soh.SalesOrderID      # Entire logic is in the RETURN statement itself
			, soh.OrderDate
			, soh.PurchaseOrderNumber
			from sales.SalesOrderHeader as soh
			where soh.CustomerID= @CustomerID AND soh.PurchaseOrderNumber IS NOT NULL
			order by soh.OrderDate DESC, soh.SalesOrderID DESC
		);

select * from sales.GetLastOrdersForCustomers(29825,3)

---------------------------------------
# EXAMPLE 02 : Write query showing Top 3 currency names with their sum of sales. Display currency name and salesamount
CREATE or ALTER FUNCTION udf_GetTopCurrency( @NumberOfCurrency int)
RETURNS TABLE
AS
RETURN (
	SELECT TOP(@NumberOfCurrency) 
		  fis.CurrencyKey
		, dc.CurrencyName
		, ROUND(SUM(fis.SalesAmount),2) AS TotalSales   
	FROM dbo.FactInternetSales AS fis
	LEFT JOIN dbo.DimCurrency AS dc
		ON fis.CurrencyKey=dc.CurrencyKey
	GROUP BY fis.CurrencyKey, dc.CurrencyName
	ORDER BY TotalSales DESC
	);

SELECT * FROM udf_GetTopCurrency(3)

# NOTE:  
# 1. In an inline TVF, all returned columns must have valid names. SUM(fis.SalesAmount) without an alias will become “No column name” and fails in a function definition.
# 2. LEFT JOIN + GROUP BY may produce a NULL “currency” group if there are currencies in fis but not in dc. Instead Use INNER JOIN.
# 3. ORDER BY inside a TVF doesn’t guarantee output order to the caller. Even though SQL Server allows ORDER BY when used with TOP, the outer query that consumes the function 
# must specify its own ORDER BY if you need deterministic ordering. ORDER BY here is helping the engine choose which top rows to return, not the final display order.
# Best practice: Always ORDER BY in the outer SELECT when you call the function'.
# 4. HOW TO CALL A I-TVF : 
# SELECT dbo.TopCurrencies(3); INCORRECT. We cannot call it like we do for a SCALAR FUNCTION. SQL doesn’t know how to show a table as a single value
# SELECT * FROM dbo.TopCurrencies(3); CORRECT

## MULTIVALUED TABLE VALUED FUNCTION

In [ ]:
# EXAMPLE 01
CREATE OR ALTER FUNCTION dbo.GetDateRange( @StartDate DATE, @NumberOfDays INT)
RETURNS @DateList TABLE (Position int, DateValue DATE)
AS 
BEGIN
	DECLARE @counter int = 0
	WHILE (@counter < @NumberOfDays) 
	BEGIN
		INSERT INTO @DateList VALUES (@counter + 1, DATEADD(day,@counter,@StartDate));
		set @counter +=1
	END;
	RETURN;
END;
GO

select * from dbo.GetDateRange('2009-12-31',10)

# STORED PROCEDURE and DYNAMIC QUERY

## Declaring and assigning variables in SQL Server

In [ ]:
# WAY 01
DECLARE @MyVar INT;
SET @MyVar = 100;

# WAY 02
DECLARE @Country NVARCHAR(50) = N'India'

# WAY 03
DECLARE @SalesAmount MONEY;

SELECT @SalesAmount = SUM(SalesAmount)
FROM FactInternetSales
WHERE SalesTerritoryKey = 9;

select @salesAmount
#NOTE: SET vs SELECT — difference
# 1. SET
# Assigns only one variable
# Throws an error if query returns more than one row

# 2. SELECT
# Can assign multiple variables
# If multiple rows returned → assigns the last row’s value. If the query returns 3 rows, SQL Server will:
# take Row 1 → assign value
# take Row 2 → assign value
# take Row 3 → assign value → @x ends up with this value
# If no rows returned → variable stays unchanged (no error)

## OPTIONAL : DYNAMIC SQL
Dynamic SQL is a technique where SQL statements are built and executed at runtime instead of being fixed. It allows queries to change based on user input, conditions, or program logic, making applications more flexible and powerful when working with databases.

It is useful when:
1. Table names or column names are not known in advance
2. Filters depend on user input
3. Queries need to change dynamically

In [ ]:
# Example without dynamic SQL
# Goal: For a given CustomerID, get the latest SalesOrderNumber, return it as an output, and then use it to fetch OrderDate and TotalDue.

DECLARE @SalesOrderNumber NVARCHAR(25);
DECLARE @CustomerID INT = 22276;
# -- Assign latest order number to variable
SELECT @SalesOrderNumber = MAX(SalesOrderNumber)
FROM Sales.SalesOrderHeader
WHERE CustomerID = @CustomerID;

SELECT OrderDate, TotalDue
FROM Sales.SalesOrderHeader
WHERE SalesOrderNumber = @SalesOrderNumber;

In [ ]:
# With Dynamic SQL (parameterized via sp_executesql)

DECLARE @SQLString        NVARCHAR(500);
DECLARE @ParmDefinition   NVARCHAR(500);
DECLARE @SalesOrderNumber NVARCHAR(25);
DECLARE @CustomerID       INT = 22276;

# -- Build the dynamic SQL text
SET @SQLString = N'
    SELECT @SalesOrderOUT = MAX(SalesOrderNumber)
    FROM Sales.SalesOrderHeader
    WHERE CustomerID = @CustomerID'  
# Note: salesOrderOUT is still not a local variable. 

# -- Declare the parameters that the dynamic SQL expects
SET @ParmDefinition = N'
    @CustomerID   INT,
    @SalesOrderOUT NVARCHAR(25) OUTPUT';

# -- Execute and capture the output into a local variable
EXEC sp_executesql
    @SQLString,
    @ParmDefinition,
    @CustomerID = @CustomerID,
    @SalesOrderOUT = @SalesOrderNumber OUTPUT;

# -- Use the output
SELECT OrderDate, TotalDue
FROM Sales.SalesOrderHeader
WHERE SalesOrderNumber = @SalesOrderNumber;

## STORED PROCEDURE

<!-- Advantages of Stored Subprograms over Independent SQL Statements: -->
1. Performance Improvement :
a) Reduced Network Traffic
Independent SQL statements must travel from the application to the database every time, increasing network round-trips.
Stored procedures reside in the database, so only the procedure call is sent — not the entire SQL text.
b) Execution Plan Reuse
Stored procedures are precompiled and optimized by SQL Server. When executed again: SQL Server reuses the already cached execution plan.
Saves time on parsing, compiling, and optimizing SQL every time. This gives faster, consistent performance compared to ad‑hoc SQL statements.

2. Controlled Access
Applications can be given permission to execute the stored procedure without granting access to the underlying tables.
Example : GRANT EXECUTE ON usp_TransferFunds TO AppUser;
This protects tables from: direct updates, accidental data corruption, unauthorized reads/writes

3. Prevents SQL Injection (when properly parametrized)
Stored procedures use parameters, not string concatenation, which reduces injection attack surfaces.

4. Improved Deployment
When moving from DEV → QA → PROD: you deploy stored procedures as database objects, no need to update every application codebase.
This simplifies large enterprise deployments.


<!-- Types of stored procedures: -->
1. User-defined
2. Temporary (stored in tempDB)
3. System (Included with DB Engine,  start with the prefix 'sp_')

<!-- How to create: -->
1. Using the ssms
1.1 GO to Programmability. Right click it and select STORED PROCEDURE. A new query window opens with a template for the stored procedure
1.2 On the Query menu, select Specify Values for Template Parameters
1.3 In the Query Editor, replace the SELECT statement with the query for your procedure
1.4 To test the syntax, on the Query menu, select Parse. Correct any errors
1.5 Select Execute from the toolbar. The procedure is created as an object in the database

2. Using SQL Query
<!-- Example -->
CREATE PROCEDURE SalesLT.uspGetCustomerCompany1
    @LastName nvarchar(50),
    @FirstName nvarchar(50)
AS   

    SET NOCOUNT ON;
    SELECT FirstName, LastName, CompanyName
    FROM SalesLT.Customer
    WHERE FirstName = @FirstName AND LastName = @LastName;
GO

In [ ]:
# EXAMPLE 01 : SP WITHOUT PARAMETERS
CREATE OR ALTER	PROCEDURE usp_TerritoryWiseSales
AS 
BEGIN
	SELECT fis.SalesTerritoryKey, dst.SalesTerritoryCountry, SUM(fis.salesAmount) AS 'Sales'
	FROM FactInternetSales as fis
	JOIN DimSalesTerritory as dst ON fis.SalesTerritoryKey = dst.SalesTerritoryKey
	group by fis.SalesTerritoryKey,  dst.SalesTerritoryCountry
	ORDER BY 'Sales' DESC
END;

EXEC usp_TerritoryWiseSales

In [ ]:
# QUESTION 01 : Can we use select to query a SP? 
NO - SP are executable programs, not table expressions.

# QUESTION 02 :So how can you query the result of a stored procedure?
Use a temp Table and insert results of SP and then query
#example
CREATE TABLE #temp   ( 
    SalesOrderID INT,
    OrderDate    DATE,
    TotalDue     MONEY
);

INSERT INTO #temp
EXEC dbo.GetRecentOrders @CustomerID = 123;

SELECT * FROM #temp;

# NOTE: The # symbol in front of a table name indicates that it is a temporary table. It means you are creating a local temporary table.
# 1. Temporary tables only exist in tempdb (not in your current database). tempdb is a built-in system database that SQL Server creates and manages 
# automatically every time the SQL Server service starts. You never create it manually.
# 2. The temp table exists until you drop it (DROP TABLE #temp) or your session ends

In [ ]:
# EXAMPLE 02 : SP WITH PARAMETERS
CREATE OR ALTER PROCEDURE uspGetCustomerCompany1
    @FN nvarchar(50),
	@LN nvarchar(50)
AS   

    SET NOCOUNT ON;  
# SET NOCOUNT ON tells SQL Server not to send “row count” messages back to the client after each INSERT, UPDATE, DELETE, or SELECT. Normally, SQL Server 
# sends messages like:(1 row affected) etc. These messages are not result sets, but they are extra network traffic and can slow down large SPs.
    SELECT FirstName, LastName, GeographyKey
    FROM DimCustomer
    WHERE FirstName = @FN AND LastName = @LN;
GO

EXEC uspGetCustomerCompany1 'Jon', 'Yang'
EXEC uspGetCustomerCompany1 @LN='Yang', @FN='Jon'

### Exception Handling

In [ ]:
# EXAMPLE : SP for Fund Transfer with Validation, Error Handling, and Transactions
# Objective : Build a robust stored procedure to transfer funds between two customer accounts with:
# 1. Input validation (source/target account existence)
# 2. Business rule enforcement (non-negative balances)
# 3. Transaction safety (atomic updates with TRY...CATCH, COMMIT/ROLLBACK)
# 4. Error signaling using THROW

------------------------------------------------------
# Step 1 — Setup: Table and Seed Data
CREATE TABLE CustomerData (
ID INT PRIMARY KEY,
FN NVARCHAR(50) NOT NULL,
Balance INT Check (Balance >=0)    
)
# INSERT VALUES
INSERT INTO CustomerData (ID,FN,Balance)
VALUES
(1,'A',10),
(2,'B',10)

SELECT * from CustomerData
------------------------------------------------------
# Step 2 — Iteration 01: Basic Validation with RAISERROR
# Purpose: Validate source and destination account existence before proceeding.
CREATE OR ALTER PROCEDURE UpTransferAmount
@fromID INT,
@toID INT,
@amount INT
AS
BEGIN
# Validate the FROM Account 
IF NOT EXISTS (SELECT * FROM CustomerData where ID=@fromID)
BEGIN
	RAISERROR ('FROM Account doesnot exists',16,1)
	RETURN;
END;
PRINT 'FROM Account is valid'

# Validate the TO Account
IF NOT EXISTS (SELECT * FROM CustomerData where ID=@toID)
BEGIN
	RAISERROR ('TO Account doesnot exists',16,1)
	RETURN;
END;
PRINT 'TO Account is valid'
END;

EXEC UpTransferAmount 1,2,50
EXEC UpTransferAmount 1,3,50
EXEC UpTransferAmount 10,2,50
EXEC UpTransferAmount 1,2,5000
------------------------------------------------------
# Step 3 — Introduce a Reusable UDF for ID Validation
CREATE OR ALTER FUNCTION udf_validation (@IDToValidate INT)
RETURNS INT
AS 
BEGIN
IF NOT EXISTS (SELECT * FROM CustomerData WHERE ID=@IDToValidate)
	RETURN -1;
RETURN 1;
END;
------------------------------------------------------
# Step 4 — Iteration 02: Credit/Debit Without Transaction (Demonstrates the Problem)
# Purpose: Show why a transaction is needed. If debit fails due to CHECK constraint, credit may already have been applied.
CREATE OR ALTER PROCEDURE UpTransferAmount
@fromID INT,
@toID INT,
@amount INT
AS
BEGIN
-- Validate the FROM Account 
IF dbo.udf_validation(@fromID)=-1
	THROW 51000, 'FROM Account is INVALID', 1;
PRINT 'FROM Account is valid'

-- Validate the TO Account
IF dbo.udf_validation(@toID)=-1
	THROW 51000, 'TO Account is INVALID', 1;
PRINT 'TO Account is valid'
# NOTE: RAISERROR is deprecated which means its support is no lnger available in the newer versions and THROW is used instead. 
# THROW cannot be used inside a UDF but can be used inside a SP.
UPDATE CustomerData
SET Balance= Balance + @amount
WHERE ID=@toID
PRINT 'Amount Credited to toID Account'

UPDATE CustomerData
SET Balance= Balance - @amount
WHERE ID=@fromID
PRINT 'Amount Debited from fromID Account'
END;

SELECT * FROM CustomerData
EXEC UpTransferAmount 1,2,50
# Now we observed that the on executing the stored procedure untill 
# one of the account went to zero seems no issues but executing after the debitors acoount went zero
# gives us error - 'The UPDATE statement conflicted with the CHECK constraint "CK__CustomerD__Balan__531856C7".'
# Although an error is thrown the amount is also credited to the creditors account!

------------------------------------------------------
# Step 5 — Iteration 03: Make It Atomic with TRY/CATCH + Transaction
# Purpose: Ensure both updates succeed or none do.
CREATE OR ALTER PROCEDURE UpTransferAmount
@fromID INT,
@toID INT,
@amount INT
AS
BEGIN
# -- Validate the FROM Account 
IF dbo.udf_validation(@fromID)=-1
	THROW 51000, 'FROM Account is INVALID', 1;
PRINT 'FROM Account is valid'

# -- Validate the TO Account
IF dbo.udf_validation(@toID)=-1
	THROW 51000, 'TO Account is INVALID', 1;
PRINT 'TO Account is valid'

BEGIN TRY
	PRINT 'INISDE TRY BLOCK'
	BEGIN TRANSACTION

	UPDATE CustomerData
	SET Balance= Balance + @amount
	WHERE ID=@toID
	PRINT 'Amount Credited to toID Account'


	UPDATE CustomerData
	SET Balance= Balance - @amount
	WHERE ID=@fromID
	PRINT 'Amount Debited from fromID Account'

	COMMIT TRANSACTION
	PRINT 'COMMIT DONE'
END TRY
BEGIN CATCH
	PRINT 'INSIDE CATCH'
	ROLLBACK TRANSACTION
	PRINT 'ROLLBACK...done succesfully';
END CATCH;
END;


UPDATE CustomerData
SET Balance=5

SELECT * FROM CustomerData
EXEC UpTransferAmount 1,2,1

------------------------------------------------------
# -- The below is the way a program calls a SP. 
BEGIN TRY
    PRINT 'Inside caller';
    EXEC UpTransferAmount 1, 2, 1;
    PRINT 'SUCCESS - back in caller';
    RETURN;
END TRY
BEGIN CATCH
    PRINT 'Inside CATCH of caller'
     SELECT  
        ERROR_NUMBER() AS ErrorNumber  
        ,ERROR_SEVERITY() AS ErrorSeverity  
        ,ERROR_STATE() AS ErrorState  
        ,ERROR_PROCEDURE() AS ErrorProcedure  
        ,ERROR_LINE() AS ErrorLine  
        ,ERROR_MESSAGE() AS ErrorMessage; 
END CATCH

### SP with Input/Output Parameters

In [ ]:
# EXAMPLE : SP with Input/Output Parameters
# Purpose of the Script
# Create a stored procedure Production.uspGetList that:
# 1. Accepts a product category pattern and maximum price as inputs.
# 2. Returns two OUTPUT parameters:
# @ComparePrice → echoes the input @MaxPrice for external comparison.
# @ListPrice → the maximum ListPrice of products that match the filters.
# 3. Returns a result set of products matching the category pattern and below the max price.

# Demonstrates:
# 1. Existence check with OBJECT_ID().
# 2. Parameter direction (input vs. output).
# 3. Suppressing row count messages using SET NOCOUNT ON.
# 4. Calling a stored procedure and using the output variables in T-SQL logic.
    
#  NOTE : OBJECT_ID() returns the numeric ID (INT) of a database object (table, view, procedure, function, constraint, synonym, etc.) given its name.
# If the object doesn’t exist (or you don’t have permission to “see” it), it returns NULL.
# OBJECT_ID ( ' [database_name.][schema_name.]object_name ' [ , 'object_type' ] )
# Tables: 'U' (user table), 'S' (system table), Views: 'V', Procedures: 'P'


USE AdventureWorks2022;  
GO  
# STEP 01 :  Drop the procedure if it exists (type 'P' = stored procedure)
IF OBJECT_ID ( 'Production.uspGetList', 'P' ) IS NOT NULL    
    DROP PROCEDURE Production.uspGetList;  
GO 

# STEP 02 :  Procedure Definition: Parameters and Logic
CREATE PROCEDURE Production.uspGetList
      @Product      varchar(40)          -- INPUT: subcategory name pattern (e.g., '%Bikes%')
    , @MaxPrice     money                -- INPUT: upper bound for ListPrice
    , @ComparePrice money OUTPUT         -- OUTPUT: returns a comparable price (echo of @MaxPrice)
    , @ListPrice    money OUT            -- OUTPUT: returns max ListPrice that meets the criteria
AS
BEGIN
    SET NOCOUNT ON;  
    SELECT
        p.[Name]              AS Product,
        p.ListPrice           AS [List Price]
    FROM Production.Product AS p
    JOIN Production.ProductSubcategory AS s
      ON p.ProductSubcategoryID = s.ProductSubcategoryID
    WHERE
        s.[Name] LIKE @Product
        AND p.ListPrice < @MaxPrice;
    # Populate the @ListPrice OUTPUT: greatest ListPrice under @MaxPrice for the pattern
    SET @ListPrice = (
        SELECT MAX(p.ListPrice)
        FROM Production.Product AS p
        JOIN Production.ProductSubcategory AS s
          ON p.ProductSubcategoryID = s.ProductSubcategoryID
        WHERE
            s.[Name] LIKE @Product
            AND p.ListPrice < @MaxPrice
    );

    # Populate the @ComparePrice OUTPUT (simply reflect the requested @MaxPrice)
    SET @ComparePrice = @MaxPrice;
END
GO

# STEP 03 :  Executing the Procedure and Using Outputs
DECLARE @ComparePrice money, @Cost money;

EXECUTE Production.uspGetList
      @Product      = '%Bikes%'
    , @MaxPrice     = 700
    , @ComparePrice = @ComparePrice OUT
    , @ListPrice    = @Cost OUTPUT;

# Use the returned outputs in client logic
IF @Cost <= @ComparePrice
BEGIN
    PRINT 'These products can be purchased for less than $'
          + RTRIM(CAST(@ComparePrice AS varchar(20))) + '.';
END
ELSE
BEGIN
    PRINT 'The prices for all products in this category exceed $'
          + RTRIM(CAST(@ComparePrice AS varchar(20))) + '.';
END

## Comparison between SP, VIEW, UDFs
1. View: A named, reusable SELECT (no parameters). Read‑only abstraction over one or more tables.
2. Inline TVF (iTVF): A parameterized single‑SELECT that returns a table. Think of it as a parameterized view.
3. Multi‑statement TVF (mTVF): Returns a table variable populated via procedural steps (multiple statements).
4. Stored Procedure (Proc / SP): A programmable unit that can do almost anything: 
Accept parameters, 
return result sets, 
run DML (INSERT/UPDATE/DELETE/MERGE), 
manage transactions, 
use dynamic SQL, error handling, output parameters, etc.

# TABLE PARTITIONING
What is Table Partitioning?
Partitioning splits a large table (or index) horizontally into multiple smaller, manageable partitions, based on a partitioning column (e.g., a date). All partitions logically form one table, but physically they can live on different filegroups, enabling:
1. Better manageability (e.g., fast archiving via SWITCH),
2. Potentially better performance for range queries and maintenance,
3. Independent index rebuilds per partition.

Partitioning uses:
1. Partition Function – maps values to partitions by defining boundary values and side (RANGE LEFT/RIGHT).
2. Partition Scheme – maps each partition produced by the function to a filegroup.
-- RANGE RIGHT means a boundary value belongs to the right-hand partition (the higher partition number).

In [ ]:
# EXAMPLE
# Objectives : 
# 1. Creating a database and a partitioned table on a single filegroup (PRIMARY).
# 2. Creating multiple filegroups and placing partitions across them (for storage distribution).
# 3. Inserting rows and observing which partition they land in.
# 4. Using DMVs (Dynamic management views) to inspect partition metadata, boundary values, partitioning column, and row counts.
# 5. MERGE and SPLIT operations on the partition function (to collapse/split ranges dynamically).

/* =========================
   1) Create a fresh database
   ========================= */
IF DB_ID('PartitionTest') IS NULL
    CREATE DATABASE PartitionTest;
GO
USE PartitionTest;
GO

/* ==========================================
   2) (Option A) Partition on a single filegroup
   All partitions map to PRIMARY
   ========================================== */

-- Clean up if objects already exist (idempotent)
IF OBJECT_ID('dbo.PartitionTable', 'U') IS NOT NULL
    DROP TABLE dbo.PartitionTable;
IF EXISTS (SELECT 1 FROM sys.partition_schemes WHERE name = 'myRangePS1')
    DROP PARTITION SCHEME myRangePS1;
IF EXISTS (SELECT 1 FROM sys.partition_functions WHERE name = 'myRangePF1')
    DROP PARTITION FUNCTION myRangePF1;
GO

-- Partition function: RANGE RIGHT with 3 boundaries ⇒ 4 partitions
-- Partitions:
--   P1: < '2022-04-01'
--   P2: >= '2022-04-01' AND < '2022-05-01'
--   P3: >= '2022-05-01' AND < '2022-06-01'
--   P4: >= '2022-06-01'
CREATE PARTITION FUNCTION myRangePF1 (datetime2(0))
AS RANGE RIGHT FOR VALUES ('2022-04-01', '2022-05-01', '2022-06-01');
GO

-- Scheme maps all partitions to PRIMARY (single FG)
CREATE PARTITION SCHEME myRangePS1
AS PARTITION myRangePF1
ALL TO ([PRIMARY]);
GO

-- Partitioned table on PRIMARY (aligned on col1)
-- PRIMARY KEY defaults to clustered and aligned
CREATE TABLE dbo.PartitionTable
(
    col1 datetime2(0) NOT NULL PRIMARY KEY,
    col2 char(10)     NOT NULL
) ON myRangePS1(col1);
GO

/* ====================================================
   3) (Option B) Partition across multiple filegroups
   Uncomment to try — requires dropping the table first
   ==================================================== */
/*
-- Add filegroups
ALTER DATABASE PartitionTest ADD FILEGROUP test1fg;
ALTER DATABASE PartitionTest ADD FILEGROUP test2fg;
ALTER DATABASE PartitionTest ADD FILEGROUP test3fg;
ALTER DATABASE PartitionTest ADD FILEGROUP test4fg;

-- Add files (ensure the folder exists and SQL service has rights)
ALTER DATABASE PartitionTest ADD FILE
( NAME = partitiontest1, FILENAME = 'C:\SSMS_Partition_tables\partitiontest1.ndf', SIZE = 5MB, FILEGROWTH = 5MB ) TO FILEGROUP test1fg;
ALTER DATABASE PartitionTest ADD FILE
( NAME = partitiontest2, FILENAME = 'C:\SSMS_Partition_tables\partitiontest2.ndf', SIZE = 5MB, FILEGROWTH = 5MB ) TO FILEGROUP test2fg;
ALTER DATABASE PartitionTest ADD FILE
( NAME = partitiontest3, FILENAME = 'C:\SSMS_Partition_tables\partitiontest3.ndf', SIZE = 5MB, FILEGROWTH = 5MB ) TO FILEGROUP test3fg;
ALTER DATABASE PartitionTest ADD FILE
( NAME = partitiontest4, FILENAME = 'C:\SSMS_Partition_tables\partitiontest4.ndf', SIZE = 5MB, FILEGROWTH = 5MB ) TO FILEGROUP test4fg;

-- Recreate function & scheme across filegroups
IF OBJECT_ID('dbo.PartitionTable', 'U') IS NOT NULL DROP TABLE dbo.PartitionTable;
IF EXISTS (SELECT 1 FROM sys.partition_schemes WHERE name = 'myRangePS1') DROP PARTITION SCHEME myRangePS1;
IF EXISTS (SELECT 1 FROM sys.partition_functions WHERE name = 'myRangePF1') DROP PARTITION FUNCTION myRangePF1;

CREATE PARTITION FUNCTION myRangePF1 (datetime2(0))
AS RANGE RIGHT FOR VALUES ('2022-04-01', '2022-05-01', '2022-06-01');

CREATE PARTITION SCHEME myRangePS1
AS PARTITION myRangePF1
TO (test1fg, test2fg, test3fg, test4fg);

CREATE TABLE dbo.PartitionTable
(
    col1 datetime2(0) NOT NULL PRIMARY KEY,
    col2 char(10)     NOT NULL
) ON myRangePS1(col1);
GO
*/

/* ===============================
   4) Insert sample data
   =============================== */
TRUNCATE TABLE dbo.PartitionTable;

-- Will map to partitions based on RANGE RIGHT
INSERT INTO dbo.PartitionTable VALUES ('2022-03-01','mandeep'); -- P1: < 2022-04-01
INSERT INTO dbo.PartitionTable VALUES ('2022-02-01','mandeep'); -- P1
INSERT INTO dbo.PartitionTable VALUES ('2022-04-01','Four');    -- P2: >= 2022-04-01 and < 2022-05-01
INSERT INTO dbo.PartitionTable VALUES ('2022-05-01','Five');    -- P3: >= 2022-05-01 and < 2022-06-01
INSERT INTO dbo.PartitionTable VALUES ('2022-06-01','Six');     -- P4: >= 2022-06-01
INSERT INTO dbo.PartitionTable VALUES ('2022-07-01','Seven');   -- P4
INSERT INTO dbo.PartitionTable VALUES ('2022-08-01','mandeep'); -- P4
SELECT * FROM dbo.PartitionTable ORDER BY col1;

-- Optional: see partition number for each row (via $PARTITION)
SELECT
    col1,
    col2,
    $PARTITION.myRangePF1(col1) AS PartitionNumber
FROM dbo.PartitionTable
ORDER BY col1;

/* ============================================
   5) How many rows per partition (heap/clustered)
   ============================================ */
SELECT
    p.partition_number,
    SUM(p.rows) AS rows_in_partition
FROM sys.dm_db_partition_stats AS p
WHERE p.object_id = OBJECT_ID('dbo.PartitionTable')
  AND p.index_id IN (0,1) -- 0=heap,1=clustered
GROUP BY p.partition_number
ORDER BY p.partition_number;

-- Full DMV raw view (all indexes)
SELECT * 
FROM sys.dm_db_partition_stats
WHERE object_id = OBJECT_ID('dbo.PartitionTable')
ORDER BY index_id, partition_number;

/* =====================================================
   6) Show partition metadata and boundary values (DMVs)
   ===================================================== */
-- (a) Partition boundaries tied to each partition_number
SELECT
    SCHEMA_NAME(t.schema_id) AS SchemaName,
    t.name AS TableName,
    i.name AS IndexName,
    p.partition_number,
    p.rows,
    f.name AS PartitionFunction,
    f.type_desc,
    f.boundary_value_on_right,
    r.boundary_id,
    rv.value AS BoundaryValue
FROM sys.tables AS t
JOIN sys.indexes AS i
    ON t.object_id = i.object_id
JOIN sys.partitions AS p
    ON i.object_id = p.object_id AND i.index_id = p.index_id
JOIN sys.partition_schemes AS s
    ON i.data_space_id = s.data_space_id
JOIN sys.partition_functions AS f
    ON s.function_id = f.function_id
LEFT JOIN sys.partition_range_values AS r
    ON f.function_id = r.function_id AND r.boundary_id = p.partition_number
LEFT JOIN sys.partition_range_values AS rv
    ON f.function_id = rv.function_id AND rv.boundary_id = p.partition_number
WHERE t.name = 'PartitionTable' 
      AND i.type <= 1
ORDER BY p.partition_number;

-- (b) Which column is the partitioning column?
SELECT
    t.object_id AS ObjectID,
    SCHEMA_NAME(t.schema_id) AS SchemaName,
    t.name AS TableName,
    ic.column_id AS PartitioningColumnID,
    c.name AS PartitioningColumnName,
    i.name AS IndexName
FROM sys.tables AS t
JOIN sys.indexes AS i
    ON t.object_id = i.object_id AND i.type <= 1 -- heap or clustered index
JOIN sys.partition_schemes AS ps
    ON ps.data_space_id = i.data_space_id
JOIN sys.index_columns AS ic
    ON ic.object_id = i.object_id
    AND ic.index_id = i.index_id
    AND ic.partition_ordinal >= 1 -- partitioning column(s)
JOIN sys.columns AS c
    ON t.object_id = c.object_id AND ic.column_id = c.column_id
WHERE t.name = 'PartitionTable';

-- (c) Human-readable range text per partition
SELECT
    SCHEMA_NAME(t.schema_id) AS SchemaName,
    t.name AS TableName,
    i.name AS IndexName,
    p.partition_number AS PartitionNumber,
    f.name AS PartitionFunctionName,
    p.rows AS Rows,
    rv.value AS BoundaryValue,
    CASE
        WHEN ISNULL(rv.value, rv2.value) IS NULL THEN 'N/A'
        ELSE
            CASE WHEN f.boundary_value_on_right = 0 AND rv2.value IS NULL THEN '>='
                 WHEN f.boundary_value_on_right = 0 THEN '>'
                 ELSE '>='
            END + ' ' + ISNULL(CONVERT(varchar(64), rv2.value), 'Min Value') + ' ' +
            CASE f.boundary_value_on_right WHEN 1 THEN 'and <' ELSE 'and <=' END
            + ' ' + ISNULL(CONVERT(varchar(64), rv.value), 'Max Value')
    END AS TextComparison
FROM sys.tables AS t
JOIN sys.indexes AS i
    ON t.object_id = i.object_id
JOIN sys.partitions AS p
    ON i.object_id = p.object_id AND i.index_id = p.index_id
JOIN sys.partition_schemes AS s
    ON i.data_space_id = s.data_space_id
JOIN sys.partition_functions AS f
    ON s.function_id = f.function_id
LEFT JOIN sys.partition_range_values AS rv
    ON f.function_id = rv.function_id AND p.partition_number = rv.boundary_id
LEFT JOIN sys.partition_range_values AS rv2
    ON f.function_id = rv2.function_id AND p.partition_number - 1 = rv2.boundary_id
WHERE t.name = 'PartitionTable' 
      AND i.type <= 1
ORDER BY p.partition_number;

/* ======================================
   7) MERGE a boundary (remove a partition)
   This merges the partition boundary at 2022-05-01,
   collapsing P2 & P3 (under RANGE RIGHT rules).
   ====================================== */
ALTER PARTITION FUNCTION myRangePF1()
MERGE RANGE ('2022-05-01');

-- Re-check ranges after MERGE
SELECT
    p.partition_number AS PartitionNumber,
    p.rows AS Rows,
    CASE
        WHEN ISNULL(rv.value, rv2.value) IS NULL THEN 'N/A'
        ELSE
            CASE WHEN f.boundary_value_on_right = 0 AND rv2.value IS NULL THEN '>='
                 WHEN f.boundary_value_on_right = 0 THEN '>'
                 ELSE '>='
            END + ' ' + ISNULL(CONVERT(varchar(64), rv2.value), 'Min Value') + ' ' +
            CASE f.boundary_value_on_right WHEN 1 THEN 'and <' ELSE 'and <=' END
            + ' ' + ISNULL(CONVERT(varchar(64), rv.value), 'Max Value')
    END AS TextComparison
FROM sys.tables AS t
JOIN sys.indexes AS i  ON t.object_id = i.object_id
JOIN sys.partitions AS p ON i.object_id = p.object_id AND i.index_id = p.index_id
JOIN sys.partition_schemes AS s ON i.data_space_id = s.data_space_id
JOIN sys.partition_functions AS f ON s.function_id = f.function_id
LEFT JOIN sys.partition_range_values AS rv ON f.function_id = rv.function_id AND p.partition_number = rv.boundary_id
LEFT JOIN sys.partition_range_values AS rv2 ON f.function_id = rv2.function_id AND p.partition_number - 1 = rv2.boundary_id
WHERE t.name = 'PartitionTable' 
      AND i.type <= 1
ORDER BY p.partition_number;

/* =======================================
   8) SPLIT a partition (add a new boundary)
   Before SPLIT, tell the scheme which filegroup
   to use for the new partition (NEXT USED).
   If you are on PRIMARY only, set PRIMARY.
   ======================================= */
-- Example: if you have multiple filegroups:
-- ALTER PARTITION SCHEME myRangePS1 NEXT USED test2fg;

-- If using only PRIMARY:
ALTER PARTITION SCHEME myRangePS1 NEXT USED [PRIMARY];

-- Now split at '2022-05-01' (recreate the earlier boundary)
ALTER PARTITION FUNCTION myRangePF1() SPLIT RANGE ('2022-05-01');

-- Verify again
SELECT
    p.partition_number AS PartitionNumber,
    p.rows AS Rows,
    CASE
        WHEN ISNULL(rv.value, rv2.value) IS NULL THEN 'N/A'
        ELSE
            CASE WHEN f.boundary_value_on_right = 0 AND rv2.value IS NULL THEN '>='
                 WHEN f.boundary_value_on_right = 0 THEN '>'
                 ELSE '>='
            END + ' ' + ISNULL(CONVERT(varchar(64), rv2.value), 'Min Value') + ' ' +
            CASE f.boundary_value_on_right WHEN 1 THEN 'and <' ELSE 'and <=' END
            + ' ' + ISNULL(CONVERT(varchar(64), rv.value), 'Max Value')
    END AS TextComparison
FROM sys.tables AS t
JOIN sys.indexes AS i  ON t.object_id = i.object_id
JOIN sys.partitions AS p ON i.object_id = p.object_id AND i.index_id = p.index_id
JOIN sys.partition_schemes AS s ON i.data_space_id = s.data_space_id
JOIN sys.partition_functions AS f ON s.function_id = f.function_id
LEFT JOIN sys.partition_range_values AS rv ON f.function_id = rv.function_id AND p.partition_number = rv.boundary_id
LEFT JOIN sys.partition_range_values AS rv2 ON f.function_id = rv2.function_id AND p.partition_number - 1 = rv2.boundary_id
WHERE t.name = 'PartitionTable' 
      AND i.type <= 1
ORDER BY p.partition_number;

/* =======================
   9) Cleanup (optional)
   ======================= */
/*
DROP TABLE IF EXISTS dbo.PartitionTable;
IF EXISTS (SELECT 1 FROM sys.partition_schemes WHERE name = 'myRangePS1') DROP PARTITION SCHEME myRangePS1;
IF EXISTS (SELECT 1 FROM sys.partition_functions WHERE name = 'myRangePF1') DROP PARTITION FUNCTION myRangePF1;

-- Optional: remove files/filegroups created for demo
ALTER DATABASE PartitionTest REMOVE FILE partitiontest1;
ALTER DATABASE PartitionTest REMOVE FILE partitiontest2;
ALTER DATABASE PartitionTest REMOVE FILE partitiontest3;
ALTER DATABASE PartitionTest REMOVE FILE partitiontest4;
ALTER DATABASE PartitionTest REMOVE FILEGROUP test1fg;
ALTER DATABASE PartitionTest REMOVE FILEGROUP test2fg;
ALTER DATABASE PartitionTest REMOVE FILEGROUP test3fg;
ALTER DATABASE PartitionTest REMOVE FILEGROUP test4fg;
*/

# INDEXES

What Is an Index?
An index in SQL is like the index of a book — it helps the database find data faster without scanning the entire table.

Without index → SQL must read every row → full table scan
With index → SQL jumps straight to the matching rows → index seek

-----------------------------------------------------------------
QUESTION : On which columns shall we create indexes? 
Indexes are generally created on columns most frequently searched for. We can create a separate index on each column but that would 
take extra storage and slows down INSERT/UPDATE/DELETE operations which wont be an optmised way. 
Therefore it is recommended to only create indexes on columns most frequently searched for. 
Indexes are typically created on:
1. Columns used frequently in WHERE clauses
2. Columns used for JOINs
3. Columns used for ORDER BY / GROUP BY
4. High selectivity columns (many unique values)

Avoid indexing:
1. Low-selectivity columns like Address, Gender, IsActive
2. Columns rarely used in searches
3. Very large VARCHAR columns unless necessary
-----------------------------------------------------------------

QUESTION : What is PK?
A primary key is a column or a set of columns in a database table that uniquely identifies each record. 
It ensures that no two rows have the same value for the primary key and that the value is never NULL. 
Every table can have only one primary key, but it can consist of multiple columns (known as a composite key).
Key characteristics of a primary key:
1. Uniqueness: Each value must be unique across all rows.
2. Non-nullable: Cannot contain NULL values.
3. Minimal: Uses the smallest number of columns necessary to ensure uniqueness.
4. Time-invariant: Values should not change over the lifetime of the record.
-----------------------------------------------------------------

QUESTION : why use UNIQUE separately if PK already enforces uniqueness?
PRIMARY KEY Purpose  = main identifier. Used for foreign keys.
UNIQUE Purpose = additional business rules but this is not the main identifier. Also, it allows NUll (1 NULL allowed). Multiple columns can have this constraint.

In [ ]:
# <!-- EXAMPLE :  -->
# OBJECTIVE : We take DimCustomer as our first table having PK on CustomerKey and no index on FirstName and lastName
# We also create a copy of this table as Cust. 

Select * INTO Cust FROM DimCustomer  

# NOTE : When you copy a table using SELECT INTO then the PK,FK, Unique, Check constraints,DEFAULT constraints, Indexes, triggers
# , permissions, Partitioning etc are not transferred. 


# STEP 01 : Query Plan Comparison (Before Indexing)
SELECT * from DimCustomer WHERE FirstName='Adam' 
SELECT * from Cust WHERE FirstName='Adam'
# -- Query plan shows 50% cost of each but the top query uses a clustered index and the below query uses a Table Scan. 
# -- For now we can consider these two scans to be the same

# STEP 02 : Creating a Simple Index
CREATE INDEX idx_cust_FN
ON DimCustomer (FirstName);
# Execution cost reduces (18% vs 82%)

# STEP 03 : Querying Non-Indexed Columns
SELECT * FROM DimCustomer WHERE LastName = 'Adam';
SELECT * FROM Cust WHERE LastName = 'Adam';
# Since the index is on FirstName, not LastName. SQL Server cannot use the index → performance remains the same (50% vs 50%)

# STEP 04 : 
CREATE INDEX idx_cust_LN ON DimCustomer (LastName);

SELECT * from DimCustomer WHERE LastName IN ('Adam','Alexander','Allen')
SELECT * from Cust WHERE LastName IN ('Adam','Alexander','Allen')
# Execution cost (44% vs 56%). Indexes help less here because Multiple values require multiple lookups. 


# STEP 05 : Index Benefits with AND Filters
SELECT * FROM DimCustomer WHERE FirstName='Adam' & LastName='Allen'
SELECT * from Cust WHERE FirstName='Adam' & LastName='Allen'
# Execution cost (2% vs 98%)


# STEP 06 :  Selecting Fewer Columns is Faster
SELECT *
FROM DimCustomer WHERE FirstName='Adam' & LastName='Allen';

SELECT FirstName, LastName
FROM DimCustomer WHERE FirstName='Adam' & LastName='Allen';
# Execution cost (56% vs 44%) why?
# Since the 2nd one doesnot have to find the select columns since they are already mentioned in the where filter whereas the 
# first query has to bring all rows after filter


In [ ]:
# Clustered and Non-C Indexes
When you create a table w/o a Primary Key then we call it as a Stack/Heap table. Basically every row is stacked one over the above.
Stack tables are slower for lookups (Table Scans)
Clustered are created by default on PK and only one can be created on a table. This doesnot require seperate table to be created for storing index
We can create multiple Non-C indexes instead and it require seaparate memory to store the index. 

# WORKING WITH JSON

In [ ]:
# 1. Validate JSON
SELECT ISJSON('{"name":"John", "age":30}') AS IsValidJson;

-------------------------------------------------------------------
# 2. Extract a Value from JSON
SELECT JSON_VALUE('{"name":"Jane", "city":"New York"}', '$.city') AS City;

# QUESTION ? Can i instead do it like this
# declare @aa JSON = '{"name":"Jane", "city":"New York"}'
# SELECT JSON_VALUE(@aa, '$.city') AS City;
# # ANSWER - NO, SQL odoesnot have JSON datatype. Instead parse it as string
# DECLARE @aa NVARCHAR(MAX) = N'{"name":"Jane", "city":"New York"}';
# SELECT JSON_VALUE(@aa, '$.city') AS City;  -- Returns: New York

--------------------------------------------------------------------
# 3. Parse a JSON Array into a Table

SELECT *
FROM OPENJSON('[{"id":1, "name":"Alice"}, {"id":2, "name":"Bob"}]')
WITH (
    id INT '$.id',
    name NVARCHAR(100) '$.name'
);
# OPENJSON+WITH clause lets you strongly type JSON into table columns — very useful in ETL pipelines
--------------------------------------------------------------------
# 4. Modify JSON
DECLARE @json NVARCHAR(MAX);
SET @json = '{"info": {"address": [{"town": "Belgrade"}, {"town": "Paris"}, {"town":"Madrid"}]}}';

# Update 2nd address town
SET @json = JSON_MODIFY(@json, '$.info.address[1].town', 'London');
SELECT modifiedJson = @json;

# Update 1st address town
SET @json = JSON_MODIFY(@json, '$.info.address[0].town', 'Bengaluru');
SELECT modifiedJson = @json;

# Notes
# 1. JSON_MODIFY() updates only ONE PATH at a time.
# 2. If path doesn’t exist → SQL Server creates it.

--------------------------------------------------------------------
# 5. Generate JSON from SQL Tables
USE AdventureWorks2022;
GO

SELECT 
    soh.SalesOrderNumber,
    soh.OrderDate,
    soh.TotalDue,
    (
        SELECT sod.ProductID, sod.OrderQty
        FROM Sales.SalesOrderDetail AS sod
        WHERE sod.SalesOrderID = soh.SalesOrderID
        FOR JSON PATH
    ) AS Items
FROM Sales.SalesOrderHeader AS soh
FOR JSON PATH, ROOT('OrderDetails');

SELECT * FROM Sales.Customer FOR JSON AUTO;
SELECT * FROM Sales.Customer FOR JSON AUTO, ROOT('CustomerInfo');

# AUTO mode → Quick JSON with structure based on table relationships
# PATH mode → Full control over JSON structure


# WORKING WITH XML

In [ ]:
# 1. Parsing XML Stored Inside a Table
# Example: Extracting demographic survey data

-------------------------------------------------------------------
# 2. Extract a Value from JSON
SELECT JSON_VALUE('{"name":"Jane", "city":"New York"}', '$.city') AS City;

# QUESTION ? Can i instead do it like this
# declare @aa JSON = '{"name":"Jane", "city":"New York"}'
# SELECT JSON_VALUE(@aa, '$.city') AS City;
# # ANSWER - NO, SQL odoesnot have JSON datatype. Instead parse it as string
# DECLARE @aa NVARCHAR(MAX) = N'{"name":"Jane", "city":"New York"}';
# SELECT JSON_VALUE(@aa, '$.city') AS City;  -- Returns: New York

--------------------------------------------------------------------
# 3. Parse a JSON Array into a Table

SELECT *
FROM OPENJSON('[{"id":1, "name":"Alice"}, {"id":2, "name":"Bob"}]')
WITH (
    id INT '$.id',
    name NVARCHAR(100) '$.name'
);
# OPENJSON+WITH clause lets you strongly type JSON into table columns — very useful in ETL pipelines
--------------------------------------------------------------------
# 4. Modify JSON
DECLARE @json NVARCHAR(MAX);
SET @json = '{"info": {"address": [{"town": "Belgrade"}, {"town": "Paris"}, {"town":"Madrid"}]}}';

# Update 2nd address town
SET @json = JSON_MODIFY(@json, '$.info.address[1].town', 'London');
SELECT modifiedJson = @json;

# Update 1st address town
SET @json = JSON_MODIFY(@json, '$.info.address[0].town', 'Bengaluru');
SELECT modifiedJson = @json;

# Notes
# 1. JSON_MODIFY() updates only ONE PATH at a time.
# 2. If path doesn’t exist → SQL Server creates it.

# CALLING A REST API FROM SQL SERVER

In [ ]:
# STEP 01 : Enable Required Options
EXEC sp_configure 'show advanced options', 1;
RECONFIGURE;

EXEC sp_configure 'Ole Automation Procedures', 1;
RECONFIGURE;

# STEP 02 : Call API and Parse JSON Response
DECLARE @WinHttpObject INT;
DECLARE @ResponseJsonText VARCHAR(8000);

EXEC sp_OACreate 'WinHttp.WinHttpRequest.5.1', @WinHttpObject OUT;

EXEC sp_OAMethod @WinHttpObject, 'open', NULL, 'get', 
     'https://api.exchangerate-api.com/v4/latest/INR';

EXEC sp_OAMethod @WinHttpObject, 'send';

EXEC sp_OAMethod @WinHttpObject, 'responseText', 
     @ResponseJsonText OUTPUT;

EXEC sp_OADestroy @WinHttpObject;

-- Validate JSON
IF ISJSON(@ResponseJsonText) = 1
BEGIN
    SELECT INRRates, USDRates, GBPRates 
    FROM OPENJSON(@ResponseJsonText)
    WITH (
        INRRates VARCHAR(10) '$.rates.INR',
        USDRates VARCHAR(10) '$.rates.USD',
        GBPRates VARCHAR(10) '$.rates.GBP'
    );
END


# TODOs : 

In [ ]:
# MISCELLANEUOS
## GROUP BY ROLLUP
NULL values are valid grouping keys. For GROUP BY, all NULL are treated as one group.

## GROUP BY CUBE ( )

## GROUP BY GROUPING SETS ( )

# VIEWS
## Materialised views
## System information schema views - 
https://learn.microsoft.com/en-us/sql/relational-databases/system-information-schema-views/system-information-schema-views-transact-sql?view=sql-server-ver17



In [ ]:

1. COVERING INDEX
CREATE INDEX idx_LN_MN  ON DimCustomer (LastName) INCLUDE (MiddleName)

2. Filtered Indexes - 
-- Used when no of records that are searched often are realtively fewer in count compared to the entire table.

3. Indexed views
-- Refer this link - https://www.sqlshack.com/sql-server-indexed-views/
-- NOTE : index view dont work with STRING_AGG() function and Non-deterministic functions

-- TODO : Index usage DMVs (DMVs Dynamic Management View)
--refer Microsoft 

-- TODO : Columnstore vs Rowstore indexes

-- Diff between the belwo queries is the inclusion of NULL
-- SELECT * FROM TABLE WHERE column DISTINCT FROM value ....this includes all values other then value in that column including NULLs
-- SELECT * FROM TABLE WHERE column != value...this wont include NULLs
DROP TABLE IF EXISTS #SampleTempTable;
GO
CREATE TABLE #SampleTempTable (id INT, message nvarchar(50));
INSERT INTO #SampleTempTable VALUES (null, 'hello') ;
INSERT INTO #SampleTempTable VALUES (10, null);
INSERT INTO #SampleTempTable VALUES (17, 'abc');
INSERT INTO #SampleTempTable VALUES (17, 'yes');
INSERT INTO #SampleTempTable VALUES (null, null);
GO
 
-- The results exclude all rows where id matched the value of 17.
SELECT * FROM #SampleTempTable WHERE id IS DISTINCT FROM 17;
SELECT * FROM #SampleTempTable WHERE id <> 17; -- Doesn't show NULL
DROP TABLE IF EXISTS #SampleTempTable;
GO


-- TODO : Change Data capture
-- refer : implementing CDC - https://www.domo.com/glossary/cdc-sql#:~:text=1.%20Enable%20CDC%20at%20the%20database%20level%3A
-- refer : https://estuary.dev/blog/enable-sql-server-change-data-capture/
-- refer : https://hevodata.com/learn/sql-server-cdc/

-- Finding Expensive queries
-- Refer- https://techsapphire.net/finding-expensive-queries-in-sql-server-and-performance-optimization

In [ ]:
# The below is copied as it is from the notes. Pls go thrugh accordingly

-- TRIGGERS
/*
CREATE TRIGGER trg_AfterInsert_Products
ON Products
AFTER INSERT
AS
BEGIN
    -- Log the newly inserted product into an audit table
    INSERT INTO ProductAudit (ProductID, ProductName, InsertedDate)
    SELECT ProductID, ProductName, GETDATE()
    FROM INSERTED;
END;
*/
-- Triggers are not generally recommended


-- TODO : TEMPORAL TABLES
-- REFER - https://learn.microsoft.com/en-us/sql/relational-databases/tables/temporal-tables?view=sql-server-v…
-- REFER - https://learn.microsoft.com/en-us/sql/relational-databases/tables/temporal-table-usage-scenarios?vi…

 -- QUERY STORE
 -- Monitor performance by using the Query Store
 -- REFER - https://learn.microsoft.com/en-us/sql/relational-databases/performance/monitoring-performance-by-using-the-query-store?view=sql-server-ver17
 -- TODO : Waiting query


 -- Last queries executed on the database
 -- REFER : https://learn.microsoft.com/en-us/sql/relational-databases/performance/tune-performance-with-the-query-store?view=sql-server-ver17
 SELECT TOP 10 qt.query_sql_text,
    q.query_id,
    qt.query_text_id,
    p.plan_id,
    rs.last_execution_time
FROM sys.query_store_query_text AS qt
INNER JOIN sys.query_store_query AS q
    ON qt.query_text_id = q.query_text_id
INNER JOIN sys.query_store_plan AS p
    ON q.query_id = p.query_id
INNER JOIN sys.query_store_runtime_stats AS rs
    ON p.plan_id = rs.plan_id
WHERE rs.last_execution_time > DATEADD(HOUR, -1, GETUTCDATE())
ORDER BY rs.last_execution_time DESC;

In [ ]:
# TO DOs : slowly changing dimensions

# TIPS
1. If we have to add few rows manually to a table, then go to the table in the left index and then click "Edit top 200 rows". 
-- Once you add values, the primary key will automatically autoincrement.
   
2. How to get the server name and other such details
SELECT @@SERVERNAME as 'Servername'
SELECT @@SPID, SYSTEM_USER, user as 'user'
SELECT @@VERSION

3. In BigQuery, DISTINCT (and set operators like EXCEPT) are case-sensitive by default. So 'Show' and 'show' are considered different strings and will both be kept by DISTINCT. SQL Server databases typically use case‑insensitive collations by default.
What is COLLATION? Collation = the set of rules that a database uses to compare and sort text.
A collation determines:
Case sensitivity → whether 'A' = 'a'
Accent sensitivity → 'a' vs 'á'
Sorting rules → how strings are ordered

Solution -
If you want case-insensitive behavior, normalize casing (e.g., with LOWER) on both sides 
If collation is CI, these are treated as the SAME. If CS, they are treated as DIFFERENT.

4. GO is not a SQL keyword. It is a batch separator recognized by SQL Server. SQL Server itself does NOT execute GO.
The client tool sees GO, splits the script into batches, and sends each batch separately to SQL Server.
Some commands must be the first statement in a batch, for example: CREATE PROCEDURE, CREATE FUNCTION, CREATE VIEW, CREATE TRIGGER. Without the GO, SQL Server will give errors like: CREATE PROCEDURE must be the first statement in a query batch.

GO comes handy inc ases where:
4.1 To split large scripts into batches
4.2 To force variable scope reset. Variables only live inside a single batch
   
DECLARE @x INT = 5;
GO
SELECT @x;                            # INCORRECT since After GO, everything resets.

4.3 To control script execution order

CREATE TABLE t(id INT);
GO                                    # If you forget GO, the INSERT may fail because the table isn't recognized yet.
INSERT INTO t VALUES (1);
GO   

4.4 To run something multiple times using GO n
INSERT INTO TestTable VALUES ('Hello');
GO 5                                  # Runs the insert 5 times

5. What is SQL Injection?
SQL Injection happens when user input is directly concatenated into a SQL query.
This allows an attacker to inject malicious SQL code.
Example of vulnerable code :
DECLARE @sql NVARCHAR(MAX);
SET @sql = 'SELECT * FROM Users WHERE UserName = ''' + @InputUserName + '''';
EXEC(@sql);

If someone enters: Mandeep' OR 1=1 then it becomes SELECT * FROM Users WHERE UserName = 'Mandeep' OR 1=1 .
This returns all users — a classic SQL injection attack.

How Stored Procedures Help (When Using Parameters) : Stored procedures expect parameters, not concatenated strings.
When parameters are used correctly:
SQL Server treats user input as data, never as executable code.
SQL Server does not attach user input into the SQL string.
Even if user input contains ' OR 1=1 --, it’s stored as a literal value.

CREATE PROCEDURE GetUserDetails
    @UserName NVARCHAR(100)
AS
BEGIN
    SELECT *
    FROM Users
    WHERE UserName = @UserName;
END
EXEC GetUserDetails @UserName = 'Mandeep'' OR 1=1 --';

NOTE: Stored procedures reduce SQL injection when:
You use parameters
You avoid dynamic SQL built with string concatenation
You let SQL Server handle values safely

Stored procedures DO NOT protect you if: You concatenate user input inside dynamic SQL

